In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from utils import extract_score, postprocess_llm_pred, postprocess_llm_response
import glob
tqdm.pandas()
import uuid
from utils import dashed_line
from pprint import pprint
import pandas as pd
from difflib import get_close_matches

def get_aspect_name(pred_col):
    if 'aggressiveness' in pred_col.lower():
        return 'aggressiveness'
    elif 'relevance' in pred_col.lower():
        return 'relevance'
    elif 'coherence' in pred_col.lower():
        return 'coherence'
    elif 'suitableness' in pred_col.lower():
        return 'suitableness'
    else:
        raise Exception(
            "Invalid column name"
        )
    
def rename_similar_columns(df, suffix='_dup', similarity_threshold=1.0):
    # Create a dictionary to hold the new column names
    new_column_names = {}
    
    # Get a list of existing column names
    column_names = df.columns.tolist()
    
    # Iterate through each column name
    for i, col in enumerate(column_names):
        # Find close matches for the current column name
        close_matches = get_close_matches(col, column_names, n=len(column_names), cutoff=similarity_threshold)
        
        # If there are close matches, rename them
        for match in close_matches:
            if match != col:  # Avoid renaming the same column
                new_column_name = f"{match}{suffix}"
                new_column_names[match] = new_column_name
    
    # Rename the columns in the DataFrame
    df.rename(columns=new_column_names, inplace=True)
    
    return df

Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.
/home/amey/miniconda3/envs/socionet/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


#### Correlation Analysis

In [65]:
df_anal = pd.read_csv('/home/amey/depository/cs-eval/emnlp_2024_analysis/analysis_metrics_computed.csv')
df_anal = df_anal[df_anal['predicted_counterspeech'].notna()]
print(df_anal.shape)
df_anal.head(1)

(6526, 69)


,index,hatespeech,counterspeech,predicted_counterspeech,csType,suitableness_score,relevance_score,coherence_score,aggressiveness_score,annotator,...,zs_Llama-3-8b-chat-hf_suitableness_score,zs_Llama-3-8b-chat-hf_aggressiveness_score,Mistral-7B-Instruct-v03_relevance_score,Mistral-7B-Instruct-v03_coherence_score,Mistral-7B-Instruct-v03_suitableness_score,Mistral-7B-Instruct-v03_aggressiveness_score,zs_Mistral-7B-Instruct-v03_relevance_score,zs_Mistral-7B-Instruct-v03_coherence_score,zs_Mistral-7B-Instruct-v03_suitableness_score,zs_Mistral-7B-Instruct-v03_aggressiveness_score
0,0.0,Jews have no place here. They are not a part o...,"Jews are part of most societies, why should we...",Why do you believe that jews are not part of t...,Questioning,1.666667,3.333333,2.333333,1.666667,"(amey,osho,apoorv)_mean",...,3.0,1.0,-1.0,-1.0,-1.0,-1.0,5.0,3.0,3.0,2.0


In [66]:
automated_metrics = ['bleu_1_(pred_cs, cs)',
 'bleu_2_(pred_cs, cs)',
 'bleu_3_(pred_cs, cs)',
 'bleu_4_(pred_cs, cs)',
 'rouge_1_(pred_cs, cs)',
 'rouge_2_(pred_cs, cs)',
 'rouge_l_(pred_cs, cs)',
 'meteor_score_(pred_cs, cs)',
 'bert_score_(hs, pred_cs)',
 'bert_score_(pred_cs, cs)',
 'bm25_score_(hs, pred_cs)',
 'bm25_score_(pred_cs, cs)',
 'bart_score_(hs, pred_cs)',
 'bart_score_(cs, pred_cs)',
 'bart_score_(pred_cs, cs)',
 'bart_score_(hs, pred_cs)',
 'aggressiveness_UniEval',
 'coherence_UniEval',
 'relevance_UniEval',
 'suitableness_UniEval',
 'pc_score_(hs, pred_cs)',
 'aq_score_(pred_cs)',
 'cd_score_(hs, pred_cs)',
 'pd_score(hs, pred_cs)',
 'negative_pc_score_(hs, pred_cs)',
 'toxicity_(pred_cs)',
 'obscenity_(pred_cs)',
 'identity_attack_(pred_cs)',
 'insult_(pred_cs)',
 'gpt-4_relevance_score',
 'gpt-4_aggressiveness_score',
 'gpt-4_coherence_score',
 'gpt-4_suitableness_score',
 'gpt-4-zs_relevance_score',
 'gpt-4-zs_coherence_score',
 'gpt-4-zs_aggressiveness_score',
 'gpt-4-zs_suitableness_score',
 'relevance_score',
 'aggressiveness_score',
 'coherence_score',
 'suitableness_score',
 'Llama-3-8b-chat-hf_relevance_score',
 'Llama-3-8b-chat-hf_coherence_score',
 'Llama-3-8b-chat-hf_suitableness_score',
 'Llama-3-8b-chat-hf_aggressiveness_score',
 'Llama-3-8b-chat-hf_relevance_score',
 'Llama-3-8b-chat-hf_coherence_score',
 'Llama-3-8b-chat-hf_suitableness_score',
 'Llama-3-8b-chat-hf_aggressiveness_score',
 'zs_Llama-3-8b-chat-hf_relevance_score',
 'zs_Llama-3-8b-chat-hf_coherence_score',
 'zs_Llama-3-8b-chat-hf_suitableness_score',
 'zs_Llama-3-8b-chat-hf_aggressiveness_score',
 'Mistral-7B-Instruct-v03_relevance_score',
 'Mistral-7B-Instruct-v03_coherence_score',
 'Mistral-7B-Instruct-v03_suitableness_score',
 'Mistral-7B-Instruct-v03_aggressiveness_score',
 'zs_Mistral-7B-Instruct-v03_relevance_score',
 'zs_Mistral-7B-Instruct-v03_coherence_score',
 'zs_Mistral-7B-Instruct-v03_suitableness_score',
 'zs_Mistral-7B-Instruct-v03_aggressiveness_score']



#### Correlation Analysis

In [67]:
df_anal = pd.read_csv('/home/amey/depository/cs-eval/emnlp_2024_analysis/analysis_metrics_computed.csv')
df_anal = df_anal[df_anal['predicted_counterspeech'].notna()]
print(df_anal.shape)
df_anal.head(1)

(6526, 69)


,index,hatespeech,counterspeech,predicted_counterspeech,csType,suitableness_score,relevance_score,coherence_score,aggressiveness_score,annotator,...,zs_Llama-3-8b-chat-hf_suitableness_score,zs_Llama-3-8b-chat-hf_aggressiveness_score,Mistral-7B-Instruct-v03_relevance_score,Mistral-7B-Instruct-v03_coherence_score,Mistral-7B-Instruct-v03_suitableness_score,Mistral-7B-Instruct-v03_aggressiveness_score,zs_Mistral-7B-Instruct-v03_relevance_score,zs_Mistral-7B-Instruct-v03_coherence_score,zs_Mistral-7B-Instruct-v03_suitableness_score,zs_Mistral-7B-Instruct-v03_aggressiveness_score
0,0.0,Jews have no place here. They are not a part o...,"Jews are part of most societies, why should we...",Why do you believe that jews are not part of t...,Questioning,1.666667,3.333333,2.333333,1.666667,"(amey,osho,apoorv)_mean",...,3.0,1.0,-1.0,-1.0,-1.0,-1.0,5.0,3.0,3.0,2.0


In [68]:
automated_metrics = [
    'bleu_1_(pred_cs, cs)',
    'bleu_2_(pred_cs, cs)', 
    'bleu_3_(pred_cs, cs)', 
    'bleu_4_(pred_cs, cs)', 
    'rouge_1_(pred_cs, cs)', 
    'rouge_2_(pred_cs, cs)', 
    'rouge_l_(pred_cs, cs)', 
    'meteor_score_(pred_cs, cs)', 
    'bert_score_(hs, pred_cs)', 
    'bert_score_(pred_cs, cs)', 
    'bm25_score_(hs, pred_cs)',
    'bm25_score_(pred_cs, cs)',
    'bart_score_(hs, pred_cs)',
    'bart_score_(cs, pred_cs)',
    'bart_score_(pred_cs, cs)',
    'bart_score_(hs, pred_cs)',
    'aggressiveness_UniEval',
    'coherence_UniEval',
    'relevance_UniEval',
    'suitableness_UniEval',
    'pc_score_(hs, pred_cs)',
    'aq_score_(pred_cs)',
    'cd_score_(hs, pred_cs)',
    'pd_score(hs, pred_cs)',
    'negative_pc_score_(hs, pred_cs)',
    'toxicity_(pred_cs)',
    'obscenity_(pred_cs)',
    'identity_attack_(pred_cs)',
    'insult_(pred_cs)',
    'gpt-4_relevance_score',
    'gpt-4_aggressiveness_score',
    'gpt-4_coherence_score',
    'gpt-4_suitableness_score',
    'gpt-4-zs_relevance_score',
    'gpt-4-zs_coherence_score',
    'gpt-4-zs_aggressiveness_score',
    'gpt-4-zs_suitableness_score',
    'relevance_score',
    'aggressiveness_score',
    'coherence_score',
    'suitableness_score',
    'Llama-3-8b-chat-hf_relevance_score',
    'Llama-3-8b-chat-hf_coherence_score',
    'Llama-3-8b-chat-hf_suitableness_score',
    'Llama-3-8b-chat-hf_aggressiveness_score',
    'Llama-3-8b-chat-hf_relevance_score',
    'Llama-3-8b-chat-hf_coherence_score',
    'Llama-3-8b-chat-hf_suitableness_score',
    'Llama-3-8b-chat-hf_aggressiveness_score',
    'zs_Llama-3-8b-chat-hf_relevance_score',
    'zs_Llama-3-8b-chat-hf_coherence_score',
    'zs_Llama-3-8b-chat-hf_suitableness_score',
    'zs_Llama-3-8b-chat-hf_aggressiveness_score',
    'Mistral-7B-Instruct-v03_relevance_score',
    'Mistral-7B-Instruct-v03_coherence_score',
    'Mistral-7B-Instruct-v03_suitableness_score',
    'Mistral-7B-Instruct-v03_aggressiveness_score',
    'zs_Mistral-7B-Instruct-v03_relevance_score',
    'zs_Mistral-7B-Instruct-v03_coherence_score',
    'zs_Mistral-7B-Instruct-v03_suitableness_score',
    'zs_Mistral-7B-Instruct-v03_aggressiveness_score',
    # 'GEVAL-4_relevance_score',
    # 'GEVAL-4_coherence_score',
    # 'GEVAL-4_aggressiveness_score',
    # 'GEVAL-4_suitableness_score',
    # 'GPTScore_relevance_score',
    # 'GPTScore_coherence_score',
    # 'GPTScore_aggressiveness_score',
    # 'GPTScore_suitableness_score',
]

for x in automated_metrics:
    if x not in df_anal.columns:
        print(x)
        print('-'*50)

# cols = []
# for x in df_anal.columns:
#     if x not in automated_metrics:
#         cols.append(x)

# print(',\n'.join([f"'{x}'" for x in cols]))

In [69]:
[x for x in automated_metrics]

['bleu_1_(pred_cs, cs)',
 'bleu_2_(pred_cs, cs)',
 'bleu_3_(pred_cs, cs)',
 'bleu_4_(pred_cs, cs)',
 'rouge_1_(pred_cs, cs)',
 'rouge_2_(pred_cs, cs)',
 'rouge_l_(pred_cs, cs)',
 'meteor_score_(pred_cs, cs)',
 'bert_score_(hs, pred_cs)',
 'bert_score_(pred_cs, cs)',
 'bm25_score_(hs, pred_cs)',
 'bm25_score_(pred_cs, cs)',
 'bart_score_(hs, pred_cs)',
 'bart_score_(cs, pred_cs)',
 'bart_score_(pred_cs, cs)',
 'bart_score_(hs, pred_cs)',
 'aggressiveness_UniEval',
 'coherence_UniEval',
 'relevance_UniEval',
 'suitableness_UniEval',
 'pc_score_(hs, pred_cs)',
 'aq_score_(pred_cs)',
 'cd_score_(hs, pred_cs)',
 'pd_score(hs, pred_cs)',
 'negative_pc_score_(hs, pred_cs)',
 'toxicity_(pred_cs)',
 'obscenity_(pred_cs)',
 'identity_attack_(pred_cs)',
 'insult_(pred_cs)',
 'gpt-4_relevance_score',
 'gpt-4_aggressiveness_score',
 'gpt-4_coherence_score',
 'gpt-4_suitableness_score',
 'gpt-4-zs_relevance_score',
 'gpt-4-zs_coherence_score',
 'gpt-4-zs_aggressiveness_score',
 'gpt-4-zs_suitablenes

In [70]:
# pred_cols_llm = [
# # 'prediction_gpt-4o_relevance_score',
# # 'prediction_gpt-4o_coherence_score',
# # 'prediction_gpt-4o_suitableness_score',
# # 'prediction_gpt-4o_aggressiveness_score',
# # 'prediction_meta-llama/Llama-3-8b-chat-hf_relevance_score',
# # 'prediction_meta-llama/Llama-3-8b-chat-hf_coherence_score',
# # 'prediction_meta-llama/Llama-3-8b-chat-hf_suitableness_score',
# # 'prediction_meta-llama/Llama-3-8b-chat-hf_aggressiveness_score',
# # 'zeroshot_prediction_meta-llama/Llama-3-8b-chat-hf_suitableness_score',
# # 'zeroshot_prediction_meta-llama/Llama-3-8b-chat-hf_relevance_score',
# # 'zeroshot_prediction_meta-llama/Llama-3-8b-chat-hf_coherence_score',
# # 'zeroshot_prediction_meta-llama/Llama-3-8b-chat-hf_aggressiveness_score',
# # 'zeroshot_prediction_mistralai/Mistral-7B-Instruct-v0.3_relevance_score',
# # 'zeroshot_prediction_mistralai/Mistral-7B-Instruct-v0.3_coherence_score',
# # 'zeroshot_prediction_mistralai/Mistral-7B-Instruct-v0.3_suitableness_score',
# # 'zeroshot_prediction_mistralai/Mistral-7B-Instruct-v0.3_aggressiveness_score'
# ]

# for col in pred_cols_llm:
#     print(col)
#     print(df_anal[col].value_counts().items)
#     print("-"*50)

In [71]:
rename_dict = {x:x for x in pred_cols_llm}
rename_dict

NameError: name 'pred_cols_llm' is not defined

In [ ]:
rename_dict = {
'prediction_meta-llama/Llama-3-8b-chat-hf_relevance_score': 'Llama-3_relevance_score',
 'prediction_meta-llama/Llama-3-8b-chat-hf_coherence_score': 'Llama-3_coherence_score',
 'prediction_meta-llama/Llama-3-8b-chat-hf_suitableness_score': 'Llama-3_suitableness_score',
 'prediction_meta-llama/Llama-3-8b-chat-hf_aggressiveness_score': 'Llama-3_aggressiveness_score',
 'zeroshot_prediction_meta-llama/Llama-3-8b-chat-hf_suitableness_score': 'Llama-3-zs_suitableness_score',
 'zeroshot_prediction_meta-llama/Llama-3-8b-chat-hf_relevance_score': 'Llama-3-zs_relevance_score',
 'zeroshot_prediction_meta-llama/Llama-3-8b-chat-hf_coherence_score': 'Llama-3-zs_coherence_score',
 'zeroshot_prediction_meta-llama/Llama-3-8b-chat-hf_aggressiveness_score': 'Llama-3-zs_aggressiveness_score',
 'zeroshot_prediction_mistralai/Mistral-7B-Instruct-v0.3_relevance_score': 'Mistral-zs_relevance_score',
 'zeroshot_prediction_mistralai/Mistral-7B-Instruct-v0.3_coherence_score': 'Mistral-zs_coherence_score',
 'zeroshot_prediction_mistralai/Mistral-7B-Instruct-v0.3_suitableness_score': 'Mistral-zs_suitableness_score',
 'zeroshot_prediction_mistralai/Mistral-7B-Instruct-v0.3_aggressiveness_score': 'Mistral-zs_aggressiveness_score'
}

pred_cols_llm = list(rename_dict.values())
print(list(rename_dict.values()))
automated_metrics.extend(list(rename_dict.values()))

df_anal = df_anal.rename(
    columns=rename_dict
)

df_anal.head(1)

['Llama-3_relevance_score', 'Llama-3_coherence_score', 'Llama-3_suitableness_score', 'Llama-3_aggressiveness_score', 'Llama-3-zs_suitableness_score', 'Llama-3-zs_relevance_score', 'Llama-3-zs_coherence_score', 'Llama-3-zs_aggressiveness_score', 'Mistral-zs_relevance_score', 'Mistral-zs_coherence_score', 'Mistral-zs_suitableness_score', 'Mistral-zs_aggressiveness_score']


,level_0,index,hatespeech,counterspeech,predicted_counterspeech,csType,suitableness_score,relevance_score,coherence_score,aggressiveness_score,...,identity_attack_(pred_cs),insult_(pred_cs),uuid,"bart_score_(pred_cs, cs)","bart_score_(cs, pred_cs)","bart_score_(hs, pred_cs)",coherence_UniEval,relevance_UniEval,aggressiveness_UniEval,suitableness_UniEval
0,0,0.0,Jews have no place here. They are not a part o...,"Jews are part of most societies, why should we...",Why do you believe that jews are not part of t...,Questioning,1.666667,3.333333,2.333333,1.666667,...,0.026416,0.001635,91761985-3ea5-482d-b20f-d855df5909d0,-3.372657,-2.945376,-2.892705,0.92566,0.931815,0.921574,0.920361


#### Table1 - Correlation between metrics

In [20]:
from scipy.stats import spearmanr, kendalltau
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale
from scipy import stats

def calculate_correlations_with_scaling(df, col1, col2):
    """
    Calculate Spearman's rho and Kendall's tau for two columns in a DataFrame after scaling.

    Parameters:
    - df: pandas.DataFrame containing the data.
    - col1: str, name of the first column.
    - col2: str, name of the second column.

    Returns:
    - spearman_rho: Spearman's rho coefficient after scaling.
    - kendall_tau: Kendall's tau coefficient after scaling.
    """
    df[col1] = df[col1].apply(lambda x: 3 if x == -1 else x)
    df[col2] = df[col2].apply(lambda x: 3 if x == -1 else x)
    """
    # Initialize the MinMaxScaler to scale between -1 and 1
    scaler = MinMaxScaler(feature_range=(0, 1))
    
    # Scale the specified columns
    scaled_values = scaler.fit_transform(df[[col1, col2]])
    
    # Update the DataFrame with the scaled values
    # df_scaled = df
    df_scaled = pd.DataFrame(scaled_values, columns=[col1, col2])
    
    # Calculate Spearman's rho on the scaled data
    spearman_rho, spearman_p_value = spearmanr(df_scaled[col1], df_scaled[col2])
    
    # Calculate Kendall's tau on the scaled data
    kendall_tau, kendall_p_value = kendalltau(df_scaled[col1], df_scaled[col2])
    """

    x = np.array(df[col1].values.tolist())
    y = np.array(df[col2].values.tolist())
    x = minmax_scale(x)
    y = minmax_scale(y)   
    
    spearman_rho = stats.spearmanr(x,y).statistic
    kendall_tau = stats.kendalltau(x,y).statistic
    
    # print(spearman_rho, kendall_tau)

    return spearman_rho, kendall_tau


def calculate_correlation_matrices(df, automated_metrics, human_ratings):
    """
    Calculate Spearman's rho and Kendall's tau matrices for multiple automated metrics against human ratings.

    Parameters:
    - df: pandas.DataFrame containing the data.
    - automated_metrics: list of str, names of the automated metric columns.
    - human_ratings: list of str, names of the human rating columns.

    Returns:
    - spearman_df: DataFrame containing Spearman's rho values.
    - kendall_df: DataFrame containing Kendall's tau values.
    """
    spearman_results = []
    kendall_results = []
    
    # Iterate over each combination of automated metric and human rating
    for metric in automated_metrics:
        spearman_row = []
        kendall_row = []
        for rating in human_ratings:
            # spearman_rho, _ = spearmanr(df[metric], df[rating])
            # kendall_tau, _ = kendalltau(df[metric], df[rating])
            spearman_rho, kendall_tau = calculate_correlations_with_scaling(df, metric, rating)
            # print(spearman_rho, kendall_tau)
            spearman_row.append(spearman_rho)
            kendall_row.append(kendall_tau)
        
        spearman_results.append(spearman_row)
        kendall_results.append(kendall_row)
        
    # Convert the results to DataFrames
    spearman_df = pd.DataFrame(spearman_results, index=automated_metrics, columns=human_ratings)
    kendall_df = pd.DataFrame(kendall_results, index=automated_metrics, columns=human_ratings)
    
    return spearman_df, kendall_df

In [35]:
dataset_path = '/home/amey/depository/cs-eval/emnlp_2024_analysis/analysis_metrics_computed_refurbished.csv'
df_anal = pd.read_csv(dataset_path)
print(df_anal.shape)
df_anal.head(3)

(6526, 81)


,index,hatespeech,counterspeech,predicted_counterspeech,csType,suitableness_score,relevance_score,coherence_score,aggressiveness_score,annotator,...,GEVAL_Llama-3-8b-chat-hf_aggressiveness_score,GEVAL_gpt-4_coherence_score,GEVAL_Llama-3-8b-chat-hf_coherence_score,GEVAL_Mistral-7B-Instruct-v03_coherence_score,GEVAL_gpt-4_relevance_score,GEVAL_Llama-3-8b-chat-hf_relevance_score,GEVAL_Mistral-7B-Instruct-v03_relevance_score,GEVAL_gpt-4_suitableness_score,GEVAL_Mistral-7B-Instruct-v03_suitableness_score,GEVAL_Llama-3-8b-chat-hf_suitableness_score
0,0.0,Jews have no place here. They are not a part o...,"Jews are part of most societies, why should we...",Why do you believe that jews are not part of t...,Questioning,1.666667,3.333333,2.333333,1.666667,"(amey,osho,apoorv)_mean",...,1.0,1.0,4.0,3.0,5.0,4.0,4.0,3.0,2.0,2.0
1,1.0,Islam is a destructive and violent religion.,It is important to approach religious discussi...,I understand that you may have concerns about ...,Positive,2.333333,3.666667,4.333333,1.000000,"(amey,osho,apoorv)_mean",...,2.0,3.0,4.0,4.0,5.0,5.0,5.0,2.0,3.0,3.0
2,2.0,"Muslims are all terrorists, they have nothing ...",There is no evidence to suggest that terrorist...,It is important to recognize that terrorism is...,Denouncing,2.666667,3.333333,3.333333,1.000000,"(amey,osho,apoorv)_mean",...,1.0,3.0,4.0,5.0,5.0,4.0,5.0,3.0,3.0,2.0


In [70]:
# df_anal.to_csv('/home/amey/depository/cs-eval/emnlp_2024_analysis/analysis_metrics_computed_refurbished.csv', index=False)

In [77]:
traditional_metrics = [
'bleu_1_(pred_cs, cs)',
 'bleu_2_(pred_cs, cs)',
 'bleu_3_(pred_cs, cs)',
 'bleu_4_(pred_cs, cs)',
 'rouge_1_(pred_cs, cs)',
 'rouge_2_(pred_cs, cs)',
 'rouge_l_(pred_cs, cs)',
 'meteor_score_(pred_cs, cs)',
#  'bert_score_(hs, pred_cs)',
 'bert_score_(pred_cs, cs)',
#  'bart_score_(hs, pred_cs)',
#  'bart_score_(cs, pred_cs)',
#  'bart_score_(hs, pred_cs)',
 'pc_score_(hs, pred_cs)',
 'aq_score_(pred_cs)',
#  'cd_score_(hs, pred_cs)',
 'pd_score(hs, pred_cs)',
#  'negative_pc_score_(hs, pred_cs)',
 'toxicity_(pred_cs)',
#  'obscenity_(pred_cs)',
#  'identity_attack_(pred_cs)',
#  'insult_(pred_cs)'
 'bart_score_(pred_cs, cs)',
]

unieval_cols = [
    'aggressiveness_UniEval',
    'coherence_UniEval',
    'relevance_UniEval',
    'suitableness_UniEval',
]

prediction_cols = [

 'gpt-4-zs_relevance_score',
 'gpt-4-zs_coherence_score',
 'gpt-4-zs_aggressiveness_score',
 'gpt-4-zs_suitableness_score',
 'zs_Llama-3-8b-chat-hf_relevance_score',
 'zs_Llama-3-8b-chat-hf_coherence_score',
 'zs_Llama-3-8b-chat-hf_suitableness_score',
 'zs_Llama-3-8b-chat-hf_aggressiveness_score',
 'zs_Mistral-7B-Instruct-v03_relevance_score',
 'zs_Mistral-7B-Instruct-v03_coherence_score',
 'zs_Mistral-7B-Instruct-v03_suitableness_score',
 'zs_Mistral-7B-Instruct-v03_aggressiveness_score',
 'GEVAL_gpt-4_aggressiveness_score',
  'GEVAL_gpt-4_coherence_score',
  'GEVAL_gpt-4_relevance_score',
  'GEVAL_gpt-4_suitableness_score',
 'GEVAL_Llama-3-8b-chat-hf_aggressiveness_score',
 'GEVAL_Llama-3-8b-chat-hf_coherence_score',
 'GEVAL_Llama-3-8b-chat-hf_relevance_score',
 'GEVAL_Llama-3-8b-chat-hf_suitableness_score',
 'GEVAL_Mistral-7B-Instruct-v03_coherence_score',
 'GEVAL_Mistral-7B-Instruct-v03_relevance_score',
 'GEVAL_Mistral-7B-Instruct-v03_suitableness_score',
 'GEVAL_Mistral-7B-Instruct-v03_aggressiveness_score',
 'Llama-3-8b-chat-hf_relevance_score',
 'Llama-3-8b-chat-hf_coherence_score',
 'Llama-3-8b-chat-hf_suitableness_score',
 'Llama-3-8b-chat-hf_aggressiveness_score',
 'Mistral-7B-Instruct-v03_relevance_score',
 'Mistral-7B-Instruct-v03_coherence_score',
 'Mistral-7B-Instruct-v03_suitableness_score',
 'Mistral-7B-Instruct-v03_aggressiveness_score',
 'gpt-4_relevance_score',
 'gpt-4_aggressiveness_score',
 'gpt-4_coherence_score',
 'gpt-4_suitableness_score'
 ]

human_ratings = ['relevance_score', 'aggressiveness_score', 'coherence_score', 'suitableness_score']

In [65]:
# df_anal['gpt-4-zs_aggressiveness_score_reversed'] = df_anal['gpt-4-zs_aggressiveness_score'].apply(lambda x: 5-x)
# df_anal = df_anal[df_anal.annotator != '(amey,osho,apoorv)_mean'].reset_index()

In [78]:
aspect_name = 'suitableness_score'
y_pred_cols = [col for col in prediction_cols if aspect_name in col or f'{aspect_name}_UniEval' in col]
y_true_cols = [col for col in human_ratings if aspect_name in col]

In [84]:
# automated_metrics = pred_cols_llm
# spearman_df, kendall_df = calculate_correlation_matrices(df_anal, y_pred_cols, y_true_cols)
# spearman_df, kendall_df = calculate_correlation_matrices(df_anal, prediction_cols, human_ratings)
# spearman_df, kendall_df = calculate_correlation_matrices(df_anal, [x for x in y_pred_cols if 'Llama' in x], [x for x in y_pred_cols if 'Llama' in x])
# spearman_df, kendall_df = calculate_correlation_matrices(df_anal, y_pred_cols + y_true_cols, y_pred_cols)
# spearman_df, kendall_df = calculate_correlation_matrices(df_anal, traditional_metrics + y_pred_cols, y_true_cols)
spearman_df, kendall_df = calculate_correlation_matrices(df_anal, unieval_cols, human_ratings)


In [85]:
spearman_df, kendall_df = calculate_correlation_matrices(df_anal, unieval_cols, human_ratings)
spearman_df = spearman_df.rename_axis('metric')
spearman_df['metric'] = spearman_df.index
spearman_df.head(500)

,relevance_score,aggressiveness_score,coherence_score,suitableness_score,metric
metric,,,,,
aggressiveness_UniEval,0.180438,0.034082,0.130518,0.203661,aggressiveness_UniEval
coherence_UniEval,0.181532,-0.045066,0.175622,0.198070,coherence_UniEval
relevance_UniEval,0.196517,-0.055376,0.185655,0.208016,relevance_UniEval
suitableness_UniEval,0.204487,-0.047551,0.193047,0.212462,suitableness_UniEval


In [86]:
spearman_df.index

Index(['aggressiveness_UniEval', 'coherence_UniEval', 'relevance_UniEval',
       'suitableness_UniEval'],
      dtype='object', name='metric')

In [88]:
# spearman_df.to_csv(f'/home/amey/depository/cs-eval/results/spearman_{aspect_name}.csv', index=False)
spearman_df.to_csv(f'/home/amey/depository/cs-eval/results/spearman_unieval.csv', index=False)

##### Kendal Tau

In [89]:
kendall_df['metric'] = kendall_df.index
kendall_df

,relevance_score,aggressiveness_score,coherence_score,suitableness_score,metric
aggressiveness_UniEval,0.131509,0.024440,0.092708,0.152532,aggressiveness_UniEval
coherence_UniEval,0.131973,-0.034409,0.127004,0.147499,coherence_UniEval
relevance_UniEval,0.142907,-0.042309,0.134498,0.154863,relevance_UniEval
suitableness_UniEval,0.148751,-0.036423,0.139487,0.158593,suitableness_UniEval


In [90]:
# kendall_df.to_csv(f'/home/amey/depository/cs-eval/results/kendalltau_{aspect_name}.csv', index=False)
kendall_df.to_csv(f'/home/amey/depository/cs-eval/results/kendalltau_unieval.csv', index=False)

#### Table2 - System level ranking accuracy / NDCG score

In [90]:
df_human_eval = pd.read_csv('/home/amey/depository/cs-eval/data/annotations/human_eval.csv')
print(df_human_eval.shape)
df_human_eval.head(1)

(368, 5)


,hatespeech,csType,counterspeech,predicted_counterspeech,source
0,Above all Islam is a religion of conquest.,Denouncing,"No, islam is a religion of 1.6 billion people ...",islam only a religion of per just wrong. peace...,GPS


In [91]:
df_human_eval = df_human_eval[df_human_eval.notna()]

In [92]:
df = pd.read_csv('//home/amey/NLG-metrics/annotations_final_for_gods_sake.csv_metrics_calculated.csv')
df.head(1)

/tmp/ipykernel_141295/824629732.py:1: DtypeWarning: Columns (6,7,41,42,43,44,45,46,47) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('//home/amey/NLG-metrics/annotations_final_for_gods_sake.csv_metrics_calculated.csv')


,level_0,hatespeech,counterspeech,predicted_counterspeech,csType,source,prediction_(prompt_aggressiveness_score)_(gpt-4),prediction_(prompt_relevance_score)_(gpt-4),uuid,prediction_(prompt_aggressiveness_score)_(gpt3.5-turbo),...,GEVAL-4_relevance_score,GEVAL-4_coherence_score,GEVAL-4_aggressiveness_score,GEVAL-4_suitableness_score,GPTScore_relevance_score,GPTScore_coherence_score,GPTScore_aggressiveness_score,GPTScore_suitableness_score,"bleu_3_(pred_cs, cs)","bleu_4_(pred_cs, cs)"
0,3,Minorities vote to strip wealth and power from...,This is incorrect. jews could never be able to...,People's right to vote is not determined by ra...,Informative,dialoGPT,{ \n'Aggressiveness': '1'\n},{ \n'Relevance': '5'\n},16229ec9-79d2-4b81-86aa-7a81335584b2,NaN,...,5,2.0,1,3,5,1.0,2,2,0.00249,0.001259


In [106]:
df_ = df_human_eval.merge(df, on=['hatespeech','csType','counterspeech','predicted_counterspeech','source'])
# assert df_.shape[0] == df_human_eval.shape[0]
df_[['hatespeech','csType','counterspeech','predicted_counterspeech','source','uuid']]

# for i in range(df_human_eval.shape[0]):
#     match = df[
#         (df['hatespeech'] == df_human_eval.iloc[i]['hatespeech']) &
#         (df['csType'] == df_human_eval.iloc[i]['csType']) &
#         (df['counterspeech'] == df_human_eval.iloc[i]['counterspeech']) &
#         (df['predicted_counterspeech'] == df_human_eval.iloc[i]['predicted_counterspeech']) &
#         (df['source'] == df_human_eval.iloc[i]['source'])
#     ]
#     # assert match.shape[0] == 1
#     if match.shape[0] < 1:
#         print(i)

,hatespeech,csType,counterspeech,predicted_counterspeech,source,uuid
0,Above all Islam is a religion of conquest.,Denouncing,"No, islam is a religion of 1.6 billion people ...",islam only a religion of per just wrong. peace...,GPS,944ec738-ee5e-40d6-8493-93c1eb8b403e
1,Above all Islam is a religion of conquest.,Denouncing,"No, islam is a religion of 1.6 billion people ...",Is it fair to judge an entire religious group ...,QUARC,b0e53c71-0567-4052-971c-d777b9fcdc1a
2,Above all Islam is a religion of conquest.,Denouncing,"No, islam is a religion of 1.6 billion people ...",I must strongly denounce your offensive statem...,GPT3.5_ZS,b0449292-e5b5-4e0a-b36e-8eaabc0d3915
3,Above all Islam is a religion of conquest.,Denouncing,"No, islam is a religion of 1.6 billion people ...",Generalizing about Muslims based on the action...,GPT3.5_FS,NaN
4,Above all Islam is a religion of conquest.,Denouncing,"No, islam is a religion of 1.6 billion people ...",Stereotyping Muslims based on the actions of a...,dialoGPT,NaN
...,...,...,...,...,...,...
305,immigrants dont belong here they are the sourc...,Informative,You can't blame immigrants for all the problem...,There is no such thing as a mongrelisation of ...,GPS,b700bfe8-1962-4f35-bdbd-eb21cdd0132d
306,immigrants dont belong here they are the sourc...,Informative,You can't blame immigrants for all the problem...,Many immigrants come here seeking a better lif...,QUARC,6fbe44d4-0dcc-49bd-bec1-8a2f77724311
307,immigrants dont belong here they are the sourc...,Informative,You can't blame immigrants for all the problem...,"Actually, immigrants contribute significantly ...",dialoGPT,f1fecffd-f101-47fd-8ced-1c90ab9bb2d9
308,immigrants dont belong here they are the sourc...,Informative,You can't blame immigrants for all the problem...,I understand that you may have concerns about ...,GPT3.5_ZS,bca91e41-aa95-436b-bcc3-1f6c0760124b


In [109]:
df_['uuid'].isna().sum()

49

In [111]:
df_[['hatespeech','csType','counterspeech','predicted_counterspeech','source','uuid']].to_csv('/home/amey/depository/cs-eval/data/annotations/human_eval_formatted.csv', index=False)

In [54]:
# df_human_eval['predicted_counterspeech'].value_counts()

In [113]:
df_[['hatespeech','csType','counterspeech','predicted_counterspeech','source','uuid']].isna().sum()

hatespeech                  0
csType                      0
counterspeech               0
predicted_counterspeech     0
source                      0
uuid                       49
dtype: int64

#### Table3 - Human EVAL table

In [374]:
# df_human_annotation = pd.read_csv('/home/amey/depository/cs-eval/data/annotations/Human Evaluation - EMNLP 2024 - annotations (1).csv')
# df_human_annotation.shape
# # df_human_annotation['Rank'].isna().sum()
# df_human_annotation = df_human_annotation[df_human_annotation['Rank'].notna()]
# df_human_annotation = df_human_annotation.rename(columns={'Rank': 'rank_human_annotation'})
# df_human_annotation.head(6)
# assert df_human_annotation.shape[0] == 310

In [383]:
def combine_rankings(rankings_a, rankings_b):
    # Convert ranks to scores
    scores_a = [5 - rank + 1 for rank in rankings_a]
    scores_b = [5 - rank + 1 for rank in rankings_b]

    # Calculate the average score for each item
    average_scores = [(score_a + score_b) / 2 for score_a, score_b in zip(scores_a, scores_b)]

    # Rank the items based on the average scores
    final_ranks = sorted(((score, rank) for rank, score in enumerate(average_scores)), reverse=True)
    final_ranks = [rank + 1 for score, rank in final_ranks]

    return final_ranks

df_human_annotation1 = pd.read_csv('/home/amey/depository/cs-eval/data/human_eval/Human Evaluation - EMNLP 2024 - annotations_amey.csv')
df_human_annotation1.shape
# df_human_annotation1['Rank'].isna().sum()
df_human_annotation1 = df_human_annotation1[df_human_annotation1['Rank'].notna()]
df_human_annotation1 = df_human_annotation1.rename(columns={'Rank': 'rank_human_annotation'})
df_human_annotation1.head(6)
assert df_human_annotation1.shape[0] == 310

df_human_annotation2 = pd.read_csv('/home/amey/depository/cs-eval/data/human_eval/Human Evaluation - EMNLP 2024 - annotations_kanupriya.csv')
df_human_annotation2.shape
# df_human_annotation2['Rank'].isna().sum()
df_human_annotation2 = df_human_annotation2[df_human_annotation2['Rank'].notna()]
df_human_annotation2 = df_human_annotation2.rename(columns={'Rank': 'rank_human_annotation'})
df_human_annotation2.head(6)
assert df_human_annotation2.shape[0] == 310

combined_rankings = []

# Assuming df1 and df2 are your dataframes
for i in range(0, len(df_human_annotation1), 5):
    df1_batch = df_human_annotation1.iloc[i:i+5]['rank_human_annotation'].values.tolist()
    df2_batch = df_human_annotation2.iloc[i:i+5]['rank_human_annotation'].values.tolist()
    # Process df1_batch and df2_batch here
    final_ranks = combine_rankings(df1_batch, df2_batch)
    combined_rankings.extend(final_ranks)
    print(df1_batch, ' -- ', df2_batch, ' -- ', final_ranks)

df_human_annotation = df_human_annotation2
df_human_annotation['rank_human_annotation'] = combined_rankings

[5.0, 2.0, 1.0, 4.0, 3.0]  --  [5.0, 4.0, 1.0, 2.0, 3.0]  --  [3, 5, 4, 2, 1]
[3.0, 4.0, 2.0, 5.0, 1.0]  --  [4.0, 2.0, 1.0, 5.0, 3.0]  --  [3, 5, 2, 1, 4]
[4.0, 1.0, 2.0, 3.0, 5.0]  --  [4.0, 1.0, 2.0, 3.0, 5.0]  --  [2, 3, 4, 1, 5]
[5.0, 4.0, 3.0, 1.0, 2.0]  --  [5.0, 4.0, 2.0, 1.0, 3.0]  --  [4, 5, 3, 2, 1]
[2.0, 1.0, 3.0, 5.0, 4.0]  --  [2.0, 1.0, 3.0, 5.0, 4.0]  --  [2, 1, 3, 5, 4]
[5.0, 5.0, 3.0, 1.0, 2.0]  --  [4.0, 5.0, 3.0, 2.0, 1.0]  --  [5, 4, 3, 1, 2]
[5.0, 5.0, 5.0, 5.0, 5.0]  --  [5.0, 1.0, 5.0, 5.0, 5.0]  --  [2, 5, 4, 3, 1]
[1.0, 4.0, 5.0, 3.0, 2.0]  --  [1.0, 4.0, 5.0, 2.0, 3.0]  --  [1, 5, 4, 2, 3]
[4.0, 5.0, 1.0, 3.0, 2.0]  --  [1.0, 5.0, 2.0, 4.0, 3.0]  --  [3, 5, 1, 4, 2]
[2.0, 5.0, 4.0, 3.0, 1.0]  --  [3.0, 5.0, 4.0, 2.0, 1.0]  --  [5, 4, 1, 3, 2]
[3.0, 2.0, 5.0, 4.0, 1.0]  --  [3.0, 2.0, 5.0, 4.0, 1.0]  --  [5, 2, 1, 4, 3]
[3.0, 2.0, 5.0, 4.0, 1.0]  --  [1.0, 3.0, 5.0, 4.0, 2.0]  --  [5, 1, 2, 4, 3]
[5.0, 3.0, 4.0, 2.0, 1.0]  --  [4.0, 3.0, 5.0, 1.0, 2.0]  --  [5

In [389]:
df_human_annotation.head(1)
df_human_annotation = df_human_annotation1

In [390]:
predictions_dir = '/home/amey/depository/cs-eval/predictions/human_eval/*.csv'
main_cols = ['hatespeech', 'csType', 'counterspeech', 'predicted_counterspeech', 'source', 'uuid']

df_analysis = pd.DataFrame()

for filename in glob.glob(predictions_dir):
    model_name = filename.split('/')[-1].replace('.csv','')
    print(model_name)
    df_pred = pd.DataFrame()
    df_pred = pd.read_csv(filename)
    df_pred = df_pred.reset_index()
    pred_cols = [x for x in df_pred.columns if 'prediction_' in x and '_cleaned' in x]
    assert len(pred_cols) == 4
    assert df_pred.shape[0] == 310

    df_pred[f'prediction_{model_name}_avg_score'] = [0]*df_pred.shape[0]

    for i in range(len(pred_cols)):
        pred_col = pred_cols[i]
        aspect_name = get_aspect_name(pred_col)
        assert df_pred[pred_col].isna().sum() == 0
        df_pred.rename(columns = {
            pred_col: f"prediction_{model_name}_{aspect_name}"
        }, inplace=True)
        pred_cols[i] = f"prediction_{model_name}_{aspect_name}"
        df_pred[f'prediction_{model_name}_avg_score'] += df_pred[f"prediction_{model_name}_{aspect_name}"] if aspect_name != 'aggressiveness' else (5 - df_pred[f"prediction_{model_name}_{aspect_name}"])
    
    # Compute avg scores
    df_pred[f'prediction_{model_name}_avg_score'] = df_pred[f'prediction_{model_name}_avg_score'] / 18
    df_pred = df_pred[main_cols + pred_cols + [f'prediction_{model_name}_avg_score']]

    if df_analysis.shape[0] == 0:
        df_analysis = df_pred
    else:
        df_pred = df_pred[pred_cols + [f'prediction_{model_name}_avg_score']]
        df_analysis = pd.concat((df_analysis, df_pred), axis=1)
        assert df_analysis.shape[0] == 310

df_analysis = df_analysis.reset_index()
print(df_analysis.shape)
df_analysis.head(10)

mistralai-Mistral-7B-Instruct-v0.3_zeroshot
gpt-4o
meta-llama-Llama-3-8b-chat-hf
mistralai-Mistral-7B-Instruct-v0.3
meta-llama-Llama-3-8b-chat-hf_zeroshot
gpt-4o_zeroshot
(310, 37)


,index,hatespeech,csType,counterspeech,predicted_counterspeech,source,uuid,prediction_mistralai-Mistral-7B-Instruct-v0.3_zeroshot_relevance,prediction_mistralai-Mistral-7B-Instruct-v0.3_zeroshot_coherence,prediction_mistralai-Mistral-7B-Instruct-v0.3_zeroshot_suitableness,...,prediction_meta-llama-Llama-3-8b-chat-hf_zeroshot_relevance,prediction_meta-llama-Llama-3-8b-chat-hf_zeroshot_coherence,prediction_meta-llama-Llama-3-8b-chat-hf_zeroshot_suitableness,prediction_meta-llama-Llama-3-8b-chat-hf_zeroshot_aggressiveness,prediction_meta-llama-Llama-3-8b-chat-hf_zeroshot_avg_score,prediction_gpt-4o_zeroshot_relevance,prediction_gpt-4o_zeroshot_aggressiveness,prediction_gpt-4o_zeroshot_coherence,prediction_gpt-4o_zeroshot_suitableness,prediction_gpt-4o_zeroshot_avg_score
0,0,Above all Islam is a religion of conquest.,Denouncing,"No, islam is a religion of 1.6 billion people ...",islam only a religion of per just wrong. peace...,GPS,944ec738-ee5e-40d6-8493-93c1eb8b403e,2.0,2.0,1.0,...,2.0,2.0,2.0,2.0,0.500000,2.0,2.0,1.0,2.0,0.444444
1,1,Above all Islam is a religion of conquest.,Denouncing,"No, islam is a religion of 1.6 billion people ...",Is it fair to judge an entire religious group ...,QUARC,b0e53c71-0567-4052-971c-d777b9fcdc1a,4.0,5.0,3.0,...,4.0,4.0,2.0,1.0,0.777778,5.0,1.0,4.0,2.0,0.833333
2,2,Above all Islam is a religion of conquest.,Denouncing,"No, islam is a religion of 1.6 billion people ...",I must strongly denounce your offensive statem...,GPT3.5_ZS,b0449292-e5b5-4e0a-b36e-8eaabc0d3915,5.0,5.0,3.0,...,5.0,4.0,2.0,2.0,0.777778,5.0,1.0,5.0,3.0,0.944444
3,3,Above all Islam is a religion of conquest.,Denouncing,"No, islam is a religion of 1.6 billion people ...",Generalizing about Muslims based on the action...,GPT3.5_FS,NaN,4.0,5.0,3.0,...,4.0,4.0,3.0,1.0,0.833333,5.0,1.0,5.0,3.0,0.944444
4,4,Above all Islam is a religion of conquest.,Denouncing,"No, islam is a religion of 1.6 billion people ...",Stereotyping Muslims based on the actions of a...,dialoGPT,NaN,4.0,4.0,3.0,...,4.0,3.0,3.0,1.0,0.777778,5.0,1.0,4.0,3.0,0.888889
5,5,All Muslims should be deported.,Informative,Islam does not pose a security risk to this co...,"Actually, muslims have made significant contri...",dialoGPT,1b6f2c1c-e29d-4939-8f93-f22e3285abaa,5.0,5.0,3.0,...,5.0,4.0,2.0,1.0,0.833333,5.0,1.0,5.0,3.0,0.944444
6,6,All Muslims should be deported.,Informative,Islam does not pose a security risk to this co...,This statement is not supported by factual inf...,QUARC,5076747c-d132-41bf-988c-05655d259fbd,5.0,4.0,3.0,...,4.0,3.0,2.0,1.0,0.722222,5.0,1.0,3.0,3.0,0.833333
7,7,All Muslims should be deported.,Informative,Islam does not pose a security risk to this co...,The statement that All Muslims should be depor...,GPT3.5_ZS,485bf5cf-789a-48bc-bf21-7ed68ef426cd,5.0,5.0,3.0,...,5.0,4.0,2.0,2.0,0.777778,5.0,1.0,5.0,3.0,0.944444
8,8,All Muslims should be deported.,Informative,Islam does not pose a security risk to this co...,please not rid of oppressing muslims. muslims ...,GPS,b3acb94b-9019-45a7-9c4d-725b3184e684,4.0,2.0,2.0,...,1.0,1.0,-1.0,5.0,0.055556,1.0,4.0,1.0,-1.0,0.111111
9,9,All Muslims should be deported.,Informative,Islam does not pose a security risk to this co...,I understand that you may have concerns about ...,GPT3.5_FS,9cb2d8a6-6548-4adf-a4c2-e2eb1ff1e335,5.0,5.0,3.0,...,5.0,4.0,2.0,2.0,0.777778,5.0,1.0,5.0,3.0,0.944444


In [391]:
aggressiveness_cols = [x for x in df_analysis.columns if 'prediction_' in x and 'aggressiveness' in x]
relevance_cols = [x for x in df_analysis.columns if 'prediction_' in x and 'relevance' in x]
coherence_cols = [x for x in df_analysis.columns if 'prediction_' in x and 'coherence' in x]
suitableness_cols = [x for x in df_analysis.columns if 'prediction_' in x and 'suitableness' in x]
avg_scores_cols = [x for x in df_analysis.columns if 'prediction_' in x and 'avg' in x]

avg_scores_cols

['prediction_mistralai-Mistral-7B-Instruct-v0.3_zeroshot_avg_score',
 'prediction_gpt-4o_avg_score',
 'prediction_meta-llama-Llama-3-8b-chat-hf_avg_score',
 'prediction_mistralai-Mistral-7B-Instruct-v0.3_avg_score',
 'prediction_meta-llama-Llama-3-8b-chat-hf_zeroshot_avg_score',
 'prediction_gpt-4o_zeroshot_avg_score']

In [392]:
df_analysis[avg_scores_cols].head(1)

,prediction_mistralai-Mistral-7B-Instruct-v0.3_zeroshot_avg_score,prediction_gpt-4o_avg_score,prediction_meta-llama-Llama-3-8b-chat-hf_avg_score,prediction_mistralai-Mistral-7B-Instruct-v0.3_avg_score,prediction_meta-llama-Llama-3-8b-chat-hf_zeroshot_avg_score,prediction_gpt-4o_zeroshot_avg_score
0,0.388889,0.388889,0.5,0.444444,0.5,0.444444


In [393]:
df_analysis = df_analysis.merge(df_human_annotation)
df_analysis.shape
# assert df_analysis.shape[0] == 310
# df_analysis.head(5)

(310, 38)

In [394]:
# Remove flagged datapoint
df_analysis = df_analysis[df_analysis['hatespeech'] != 'I wish I live in a country that does not tolerate gay marriage.']
print(df_analysis.shape)

(305, 38)


In [395]:
df_analysis = df_analysis.reset_index()

In [396]:
df_analysis.head()

,level_0,index,hatespeech,csType,counterspeech,predicted_counterspeech,source,uuid,prediction_mistralai-Mistral-7B-Instruct-v0.3_zeroshot_relevance,prediction_mistralai-Mistral-7B-Instruct-v0.3_zeroshot_coherence,...,prediction_meta-llama-Llama-3-8b-chat-hf_zeroshot_coherence,prediction_meta-llama-Llama-3-8b-chat-hf_zeroshot_suitableness,prediction_meta-llama-Llama-3-8b-chat-hf_zeroshot_aggressiveness,prediction_meta-llama-Llama-3-8b-chat-hf_zeroshot_avg_score,prediction_gpt-4o_zeroshot_relevance,prediction_gpt-4o_zeroshot_aggressiveness,prediction_gpt-4o_zeroshot_coherence,prediction_gpt-4o_zeroshot_suitableness,prediction_gpt-4o_zeroshot_avg_score,rank_human_annotation
0,0,0,Above all Islam is a religion of conquest.,Denouncing,"No, islam is a religion of 1.6 billion people ...",islam only a religion of per just wrong. peace...,GPS,944ec738-ee5e-40d6-8493-93c1eb8b403e,2.0,2.0,...,2.0,2.0,2.0,0.500000,2.0,2.0,1.0,2.0,0.444444,5.0
1,1,1,Above all Islam is a religion of conquest.,Denouncing,"No, islam is a religion of 1.6 billion people ...",Is it fair to judge an entire religious group ...,QUARC,b0e53c71-0567-4052-971c-d777b9fcdc1a,4.0,5.0,...,4.0,2.0,1.0,0.777778,5.0,1.0,4.0,2.0,0.833333,2.0
2,2,2,Above all Islam is a religion of conquest.,Denouncing,"No, islam is a religion of 1.6 billion people ...",I must strongly denounce your offensive statem...,GPT3.5_ZS,b0449292-e5b5-4e0a-b36e-8eaabc0d3915,5.0,5.0,...,4.0,2.0,2.0,0.777778,5.0,1.0,5.0,3.0,0.944444,1.0
3,3,3,Above all Islam is a religion of conquest.,Denouncing,"No, islam is a religion of 1.6 billion people ...",Generalizing about Muslims based on the action...,GPT3.5_FS,NaN,4.0,5.0,...,4.0,3.0,1.0,0.833333,5.0,1.0,5.0,3.0,0.944444,4.0
4,4,4,Above all Islam is a religion of conquest.,Denouncing,"No, islam is a religion of 1.6 billion people ...",Stereotyping Muslims based on the actions of a...,dialoGPT,NaN,4.0,4.0,...,3.0,3.0,1.0,0.777778,5.0,1.0,4.0,3.0,0.888889,3.0


#### Compute Ranking by each automated metric for each model

In [402]:
model_name_mappings = {
    'gpt-4o_zeroshot': 'gpt-4o_zeroshot',
    'gpt-4o': 'gpt-4o',
    'meta-llama-Llama-3-8b-chat-hf_zeroshot': 'llama_zeroshot',
    'meta-llama-Llama-3-8b-chat-hf': 'llama',
    'mistralai-Mistral-7B-Instruct-v0.3': 'mistral',
    'mistralai-Mistral-7B-Instruct-v0.3_zeroshot': 'mistral_zeroshot',
}
# df_analysis = df_analysis.drop(columns=['level_0'])
df_analysis['uuid_ranking'] = df_analysis['csType'].apply(lambda x: str(uuid.uuid4()))

In [403]:
df_grouped = df_analysis.groupby(['hatespeech','csType'])
df_ranking = pd.DataFrame()
cols = [x for x in df_analysis.columns if 'prediction_' in x and 'avg' in x]

for group_key, group_df in df_grouped:
    # add prediction rankings
    ranking_dict = {}
    for model in model_name_mappings:
        rank_col = [x for x in cols if model in x][0]
        df_sorted = group_df.sort_values(by=rank_col, ascending=False)
        ranks = df_sorted['uuid_ranking'].values.tolist()
        if model_name_mappings[model] in ranking_dict:
            ranking_dict[model_name_mappings[model]].append(ranks)
        else:
            ranking_dict[model_name_mappings[model]] = [ranks]
    
    
    for col in traditional_metrics:
        rank_col = col
        df_sorted = group_df.sort_values(by=rank_col, ascending=False)
        ranks = df_sorted['uuid_ranking'].values.tolist()
        ranking_dict[col] = [ranks]

    # add groundtruth rankings
    df_sorted = group_df.sort_values(by='rank_human_annotation', ascending=True)
    ranks = df_sorted['uuid_ranking'].values.tolist()
    ranking_dict['human_annotation'] = [ranks]

    df_ranking = pd.concat((df_ranking,pd.DataFrame.from_dict(ranking_dict)))

print(f"Total rankings: {df_ranking.shape}")
df_ranking.head(3)

Total rankings: (61, 16)


,gpt-4o_zeroshot,gpt-4o,llama_zeroshot,llama,mistral,mistral_zeroshot,"bleu_1_(pred_cs, cs)","bleu_2_(pred_cs, cs)","bleu_3_(pred_cs, cs)","bleu_4_(pred_cs, cs)","rouge_1_(pred_cs, cs)","rouge_2_(pred_cs, cs)","rouge_l_(pred_cs, cs)","meteor_score_(pred_cs, cs)","bert_score_(pred_cs, cs)",human_annotation
0,"[6964ebf8-42e5-4818-b290-64382d4de00c, 152248d...","[d8a5dfe2-f3a8-4086-a69d-8d27597ed48a, 152248d...","[152248db-1137-4f4f-aaf6-ef788c6c2043, 6964ebf...","[6964ebf8-42e5-4818-b290-64382d4de00c, 152248d...","[d8a5dfe2-f3a8-4086-a69d-8d27597ed48a, 152248d...","[d8a5dfe2-f3a8-4086-a69d-8d27597ed48a, 152248d...","[2513b132-f5c6-4391-97a5-1f4157df97f5, d8a5dfe...","[2513b132-f5c6-4391-97a5-1f4157df97f5, d8a5dfe...","[2513b132-f5c6-4391-97a5-1f4157df97f5, d8a5dfe...","[2513b132-f5c6-4391-97a5-1f4157df97f5, d8a5dfe...","[2513b132-f5c6-4391-97a5-1f4157df97f5, 152248d...","[2513b132-f5c6-4391-97a5-1f4157df97f5, 6964ebf...","[2513b132-f5c6-4391-97a5-1f4157df97f5, d8a5dfe...","[152248db-1137-4f4f-aaf6-ef788c6c2043, 2513b13...","[d8a5dfe2-f3a8-4086-a69d-8d27597ed48a, 74703f8...","[6964ebf8-42e5-4818-b290-64382d4de00c, d8a5dfe..."
0,"[ce655b57-8a83-4e69-9efc-f8d2bc36ef62, c3ef8aa...","[c3ef8aa2-80d5-4972-9dd5-ffb7285b2025, 9c8d072...","[ce655b57-8a83-4e69-9efc-f8d2bc36ef62, c3ef8aa...","[ce655b57-8a83-4e69-9efc-f8d2bc36ef62, 9c8d072...","[ce655b57-8a83-4e69-9efc-f8d2bc36ef62, 144bb5e...","[ce655b57-8a83-4e69-9efc-f8d2bc36ef62, 144bb5e...","[9c8d0726-4b8f-4fdd-ae5a-93d7eadca40b, ce655b5...","[3dccc11d-2d4d-404d-9f65-48c8470a2233, 144bb5e...","[3dccc11d-2d4d-404d-9f65-48c8470a2233, 144bb5e...","[3dccc11d-2d4d-404d-9f65-48c8470a2233, 144bb5e...","[144bb5ed-e7cc-4ceb-a10f-9431649de51a, 3dccc11...","[ce655b57-8a83-4e69-9efc-f8d2bc36ef62, 144bb5e...","[144bb5ed-e7cc-4ceb-a10f-9431649de51a, 3dccc11...","[9c8d0726-4b8f-4fdd-ae5a-93d7eadca40b, c3ef8aa...","[3dccc11d-2d4d-404d-9f65-48c8470a2233, 144bb5e...","[9c8d0726-4b8f-4fdd-ae5a-93d7eadca40b, c3ef8aa..."
0,"[189a7b36-e646-4e25-9fba-eacdb916ea2d, 7bfd170...","[189a7b36-e646-4e25-9fba-eacdb916ea2d, 7bfd170...","[7bfd1705-fc1b-4870-b90b-6889172511fc, 33ed5c6...","[189a7b36-e646-4e25-9fba-eacdb916ea2d, 7bfd170...","[32c51b86-f2ae-45a6-8001-e0c851ec838f, 189a7b3...","[32c51b86-f2ae-45a6-8001-e0c851ec838f, 189a7b3...","[32c51b86-f2ae-45a6-8001-e0c851ec838f, 7bfd170...","[e313f5fe-f47e-4b74-889f-092372f27727, 32c51b8...","[e313f5fe-f47e-4b74-889f-092372f27727, 32c51b8...","[e313f5fe-f47e-4b74-889f-092372f27727, 32c51b8...","[e313f5fe-f47e-4b74-889f-092372f27727, 7bfd170...","[32c51b86-f2ae-45a6-8001-e0c851ec838f, 189a7b3...","[e313f5fe-f47e-4b74-889f-092372f27727, 7bfd170...","[189a7b36-e646-4e25-9fba-eacdb916ea2d, 7bfd170...","[e313f5fe-f47e-4b74-889f-092372f27727, 32c51b8...","[189a7b36-e646-4e25-9fba-eacdb916ea2d, 33ed5c6..."


In [404]:
import numpy as np
from sklearn.metrics import ndcg_score
df_ranking = df_ranking

def convert_rankings_to_relevance_scores(ranking1, ranking2):
    # relevance_y_true = [(len(ranking1)-i) for i, _ in enumerate(ranking1)]
    # relevance_y_pred = [(len(ranking2) - ranking2.index(id)) for id in ranking1]
    relevance_y_true = [i+1 for i, _ in enumerate(ranking1)]
    relevance_y_pred = [ranking2.index(id) + 1 for id in ranking1]
    return relevance_y_true, relevance_y_pred

true_col = 'human_annotation'
pred_col = 'gpt-4o_zeroshot'

y_true = df_ranking[true_col].values.tolist()
y_pred = df_ranking[pred_col].values.tolist()
y_true_rankings = [convert_rankings_to_relevance_scores(ranking1, ranking2)[0] for ranking1, ranking2 in zip(y_true, y_pred)]
y_pred_rankings = [convert_rankings_to_relevance_scores(ranking1, ranking2)[1] for ranking1, ranking2 in zip(y_true, y_pred)]

ndcg_score_list = [ndcg_score([y_true_ranking], [y_pred_ranking]) for y_true_ranking, y_pred_ranking in zip(y_true_rankings, y_pred_rankings)]
avg_ndcg = ndcg_score(y_true_rankings, y_pred_rankings)
print(f"Average NDCG score: {avg_ndcg:}")

df_ranking[f'ndcg_({true_col},{pred_col})'] = ndcg_score_list
# df_ranking.head(1)

Average NDCG score: 0.9736200056866403


In [405]:
cols_list = [
    'gpt-4o_zeroshot', 'gpt-4o', 'llama_zeroshot', 'llama', 'mistral',
       'mistral_zeroshot', 'bleu_1_(pred_cs, cs)', 'bleu_2_(pred_cs, cs)',
       'bleu_3_(pred_cs, cs)', 'bleu_4_(pred_cs, cs)', 'rouge_1_(pred_cs, cs)',
       'rouge_2_(pred_cs, cs)', 'rouge_l_(pred_cs, cs)',
       'meteor_score_(pred_cs, cs)', 'bert_score_(pred_cs, cs)'
]
for pred_col in cols_list:

    true_col = 'human_annotation'

    y_true = df_ranking[true_col].values.tolist()
    y_pred = df_ranking[pred_col].values.tolist()
    y_true_rankings = [convert_rankings_to_relevance_scores(ranking1, ranking2)[0] for ranking1, ranking2 in zip(y_true, y_pred)]
    y_pred_rankings = [convert_rankings_to_relevance_scores(ranking1, ranking2)[1] for ranking1, ranking2 in zip(y_true, y_pred)]

    ndcg_score_list = [ndcg_score([y_true_ranking], [y_pred_ranking]) for y_true_ranking, y_pred_ranking in zip(y_true_rankings, y_pred_rankings)]
    avg_ndcg = ndcg_score(y_true_rankings, y_pred_rankings)
    # print(f"Average NDCG score: {avg_ndcg:}")
    print(f"{pred_col},{avg_ndcg:}")

    # print(f"avg ndcg score: ")

    df_ranking[f'ndcg_({true_col},{pred_col})'] = ndcg_score_list

gpt-4o_zeroshot,0.9736200056866403
gpt-4o,0.9655075104336339
llama_zeroshot,0.9381903637159505
llama,0.9551361268120805
mistral,0.9508248402840414
mistral_zeroshot,0.9508248402840414
bleu_1_(pred_cs, cs),0.8854666055254281
bleu_2_(pred_cs, cs),0.7871137146880147
bleu_3_(pred_cs, cs),0.7884591439575401
bleu_4_(pred_cs, cs),0.7884591439575401
rouge_1_(pred_cs, cs),0.8696017548632476
rouge_2_(pred_cs, cs),0.8773631428241773
rouge_l_(pred_cs, cs),0.8688092160896259
meteor_score_(pred_cs, cs),0.919623199358571
bert_score_(pred_cs, cs),0.8541347549042574


In [406]:
df_ranking

,gpt-4o_zeroshot,gpt-4o,llama_zeroshot,llama,mistral,mistral_zeroshot,"bleu_1_(pred_cs, cs)","bleu_2_(pred_cs, cs)","bleu_3_(pred_cs, cs)","bleu_4_(pred_cs, cs)",...,"ndcg_(human_annotation,mistral_zeroshot)","ndcg_(human_annotation,bleu_1_(pred_cs, cs))","ndcg_(human_annotation,bleu_2_(pred_cs, cs))","ndcg_(human_annotation,bleu_3_(pred_cs, cs))","ndcg_(human_annotation,bleu_4_(pred_cs, cs))","ndcg_(human_annotation,rouge_1_(pred_cs, cs))","ndcg_(human_annotation,rouge_2_(pred_cs, cs))","ndcg_(human_annotation,rouge_l_(pred_cs, cs))","ndcg_(human_annotation,meteor_score_(pred_cs, cs))","ndcg_(human_annotation,bert_score_(pred_cs, cs))"
0,"[6964ebf8-42e5-4818-b290-64382d4de00c, 152248d...","[d8a5dfe2-f3a8-4086-a69d-8d27597ed48a, 152248d...","[152248db-1137-4f4f-aaf6-ef788c6c2043, 6964ebf...","[6964ebf8-42e5-4818-b290-64382d4de00c, 152248d...","[d8a5dfe2-f3a8-4086-a69d-8d27597ed48a, 152248d...","[d8a5dfe2-f3a8-4086-a69d-8d27597ed48a, 152248d...","[2513b132-f5c6-4391-97a5-1f4157df97f5, d8a5dfe...","[2513b132-f5c6-4391-97a5-1f4157df97f5, d8a5dfe...","[2513b132-f5c6-4391-97a5-1f4157df97f5, d8a5dfe...","[2513b132-f5c6-4391-97a5-1f4157df97f5, d8a5dfe...",...,0.962741,0.748487,0.748487,0.748487,0.748487,0.722243,0.865335,0.748487,0.762440,0.800277
0,"[ce655b57-8a83-4e69-9efc-f8d2bc36ef62, c3ef8aa...","[c3ef8aa2-80d5-4972-9dd5-ffb7285b2025, 9c8d072...","[ce655b57-8a83-4e69-9efc-f8d2bc36ef62, c3ef8aa...","[ce655b57-8a83-4e69-9efc-f8d2bc36ef62, 9c8d072...","[ce655b57-8a83-4e69-9efc-f8d2bc36ef62, 144bb5e...","[ce655b57-8a83-4e69-9efc-f8d2bc36ef62, 144bb5e...","[9c8d0726-4b8f-4fdd-ae5a-93d7eadca40b, ce655b5...","[3dccc11d-2d4d-404d-9f65-48c8470a2233, 144bb5e...","[3dccc11d-2d4d-404d-9f65-48c8470a2233, 144bb5e...","[3dccc11d-2d4d-404d-9f65-48c8470a2233, 144bb5e...",...,0.926233,0.993251,0.758173,0.758173,0.758173,0.762440,0.782513,0.811116,0.987254,0.758173
0,"[189a7b36-e646-4e25-9fba-eacdb916ea2d, 7bfd170...","[189a7b36-e646-4e25-9fba-eacdb916ea2d, 7bfd170...","[7bfd1705-fc1b-4870-b90b-6889172511fc, 33ed5c6...","[189a7b36-e646-4e25-9fba-eacdb916ea2d, 7bfd170...","[32c51b86-f2ae-45a6-8001-e0c851ec838f, 189a7b3...","[32c51b86-f2ae-45a6-8001-e0c851ec838f, 189a7b3...","[32c51b86-f2ae-45a6-8001-e0c851ec838f, 7bfd170...","[e313f5fe-f47e-4b74-889f-092372f27727, 32c51b8...","[e313f5fe-f47e-4b74-889f-092372f27727, 32c51b8...","[e313f5fe-f47e-4b74-889f-092372f27727, 32c51b8...",...,0.960957,0.921966,0.722243,0.722243,0.722243,0.875021,0.960957,0.875021,0.957321,0.722243
0,"[cccb62aa-d0a1-4290-bd8f-d3f2b9fa0f13, 0f37e3b...","[447df4d1-0454-4d4c-8862-298981e75dc8, cccb62a...","[0f37e3b4-cf8a-4f12-93d3-d8f1d1ad83c8, cccb62a...","[cccb62aa-d0a1-4290-bd8f-d3f2b9fa0f13, 0f37e3b...","[cccb62aa-d0a1-4290-bd8f-d3f2b9fa0f13, 447df4d...","[cccb62aa-d0a1-4290-bd8f-d3f2b9fa0f13, 447df4d...","[0f37e3b4-cf8a-4f12-93d3-d8f1d1ad83c8, cccb62a...","[88ce5d18-dbf8-464e-a4e6-93cb2f5ff24d, 99511e6...","[88ce5d18-dbf8-464e-a4e6-93cb2f5ff24d, 99511e6...","[88ce5d18-dbf8-464e-a4e6-93cb2f5ff24d, 99511e6...",...,0.943997,0.938577,0.758173,0.758173,0.758173,0.964070,0.938577,0.964070,0.959804,0.728992
0,"[9e858255-4ef5-4048-bfc9-e00c48cde2c2, 1217f59...","[9e858255-4ef5-4048-bfc9-e00c48cde2c2, 1217f59...","[43a01ad2-ca3a-4851-b98d-974a07a0e95f, 9e85825...","[43a01ad2-ca3a-4851-b98d-974a07a0e95f, 9e85825...","[9e858255-4ef5-4048-bfc9-e00c48cde2c2, 1217f59...","[9e858255-4ef5-4048-bfc9-e00c48cde2c2, 1217f59...","[43a01ad2-ca3a-4851-b98d-974a07a0e95f, 9942515...","[4a97a00a-ea40-41bb-be82-49ef94cf6065, 9942515...","[4a97a00a-ea40-41bb-be82-49ef94cf6065, 9942515...","[4a97a00a-ea40-41bb-be82-49ef94cf6065, 9942515...",...,0.995734,0.926233,0.758173,0.758173,0.758173,0.959804,0.959804,0.959804,0.959804,0.758173
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,"[292d3df3-3302-4b51-b3fd-a14826c11832, f4a4d84...","[f4a4d84e-c4ae-41cd-a11e-d7e684839191, 2aebbb0...","[292d3df3-3302-4b51-b3fd-a14826c11832, 2aebbb0...","[f4a4d84e-c4ae-41cd-a11e-d7e

In [364]:
# df_ranking['rank_gpt-o'] = y_pred_rankings
# x = df_ranking[['gpt-4o', 'rank_gpt-o']].explode(['gpt-4o','rank_gpt-o'])
# x = x.rename(columns={'gpt-4o': 'uuid_ranking'})
# y = df_analysis[['hatespeech', 'csType', 'uuid_ranking']].merge(x, on=['uuid_ranking'])
# assert y.shape[0] == df_analysis.shape[0]
# y

In [365]:
# y.to_csv('/home/amey/temp.csv', index=False)

In [288]:
# df_analysis.to_csv('/home/amey/depository/cs-eval/data/annotations/dataset_analysis_human_annotations.csv', index=False)

In [289]:
np.mean(ndcg_score_list)

0.8632793397934301

In [290]:
def ndcg_score(y_true, y_pred, k=None):
    """
    Compute the Normalized Discounted Cumulative Gain (NDCG) score.
    
    Parameters:
    y_true (list): A list of strings representing the true ranking of items.
    y_pred (list): A list of strings representing the predicted ranking of items.
    k (int, optional): The number of top-ranked items to consider. If None, all items are considered.
    
    Returns:
    float: The NDCG score.
    """
    from math import log2
    
    if len(y_true) != len(y_pred):
        raise ValueError("y_true and y_pred must have the same length.")
    
    # Create a dictionary to store the relevance scores
    relevance = {item: i+1 for i, item in enumerate(y_true)}
    
    # Compute the Discounted Cumulative Gain (DCG)
    dcg = 0
    for i, item in enumerate(y_pred):
        if item in relevance:
            rank = i + 1
            dcg += (relevance[item] - 1) / log2(rank + 1)
        if k is not None and rank >= k:
            break
    
    # Compute the Ideal Discounted Cumulative Gain (IDCG)
    idcg = sum(v / log2(i+2) for i, v in enumerate(range(len(y_true), 0, -1)))
    
    # Compute the NDCG score
    ndcg = dcg / idcg
    
    return ndcg

ranking1 = df_ranking.iloc[0]['human_annotation']
ranking2 = df_ranking.iloc[0]['gpt-4o_zeroshot']

print(ranking1)
print(dashed_line)
print(ranking2)

score = ndcg_score(ranking1, ranking2)
print(f"NDCG score: {score}")

['4e2db7d9-3560-4a83-bb7e-d756bede1130', 'c02a1ffc-e059-47a4-9140-4947d74c199e', 'feaaf02f-6bd1-44f2-a67a-3ec7354814ca', 'f4843a02-b80b-435c-8108-b23a8981e345', 'af361a88-2cbc-4d83-903d-e05c1014d02f']
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
['f4843a02-b80b-435c-8108-b23a8981e345', 'c02a1ffc-e059-47a4-9140-4947d74c199e', '4e2db7d9-3560-4a83-bb7e-d756bede1130', 'af361a88-2cbc-4d83-903d-e05c1014d02f', 'feaaf02f-6bd1-44f2-a67a-3ec7354814ca']
NDCG score: 0.5965134711864127


In [291]:
import numpy as np
from sklearn.metrics import ndcg_score

# Ground-truth relevance of some answers to a query
true_relevance = np.asarray([[1, 2, 3, 4, 5]])

# Predicted scores (relevance) for the answers
scores = np.asarray([[5, 5, 5, 5, 5]])

# Calculate NDCG score
# ndcg = ndcg_score(true_relevance, scores)
# print(f"NDCG score: {ndcg:.2f}")


relevance_y_true = [[(len(ranking1)-i) for i, _ in enumerate(ranking1)]]
relevance_y_pred = [[(len(ranking2) - ranking2.index(id)) for id in ranking1]]

print(relevance_y_pred, relevance_y_true)
ndcg = ndcg_score(relevance_y_true, relevance_y_pred)
print(f"NDCG score: {ndcg:.2f}")

[[3, 4, 1, 5, 2]] [[5, 4, 3, 2, 1]]
NDCG score: 0.84


In [292]:
# cols = [x for x in df_analysis.columns if 'prediction_' in x and 'avg' in x]
# cols
# model_specific_rankings = {}
# for model in model_name_mappings:
#     rank_col = [x for x in cols if model in x][0]
#     df_sorted = sample_df.sort_values(by=col, ascending=False)
#     ranks = df_sorted['uuid_ranking'].values.tolist()
#     if model_name_mappings[model] in model_specific_rankings:
#         model_specific_rankings[model_name_mappings[model]].append(ranks)
#     else:
#         model_specific_rankings[model_name_mappings[model]] = [ranks]

# df_ranking = pd.concat((df_ranking,pd.DataFrame.from_dict(model_specific_rankings)))
# df_ranking

#### Human Eval table - Just finish for Gods Sake

In [7]:
traditional_metrics = [
'bleu_1_(pred_cs, cs)',
 'bleu_2_(pred_cs, cs)',
 'bleu_3_(pred_cs, cs)',
 'bleu_4_(pred_cs, cs)',
 'rouge_1_(pred_cs, cs)',
 'rouge_2_(pred_cs, cs)',
 'rouge_l_(pred_cs, cs)',
 'meteor_score_(pred_cs, cs)',
#  'bert_score_(hs, pred_cs)',
 'bert_score_(pred_cs, cs)',
#  'bart_score_(hs, pred_cs)',
#  'bart_score_(cs, pred_cs)',
#  'bart_score_(hs, pred_cs)',
#  'pc_score_(hs, pred_cs)',
#  'aq_score_(pred_cs)',
#  'cd_score_(hs, pred_cs)',
#  'pd_score(hs, pred_cs)',
#  'negative_pc_score_(hs, pred_cs)',
#  'toxicity_(pred_cs)',
#  'obscenity_(pred_cs)',
#  'identity_attack_(pred_cs)',
#  'insult_(pred_cs)'
#  'bart_score_(pred_cs, cs)',
]

unieval_cols = [
    'aggressiveness_UniEval',
 'coherence_UniEval',
 'relevance_UniEval',
 'suitableness_UniEval',
]

prediction_cols = [
 'gpt-4-zs_relevance_score',
 'gpt-4-zs_coherence_score',
 'gpt-4-zs_aggressiveness_score',
 'gpt-4-zs_suitableness_score',
 'zs_Llama-3-8b-chat-hf_relevance_score',
 'zs_Llama-3-8b-chat-hf_coherence_score',
 'zs_Llama-3-8b-chat-hf_suitableness_score',
 'zs_Llama-3-8b-chat-hf_aggressiveness_score',
 'zs_Mistral-7B-Instruct-v03_relevance_score',
 'zs_Mistral-7B-Instruct-v03_coherence_score',
 'zs_Mistral-7B-Instruct-v03_suitableness_score',
 'zs_Mistral-7B-Instruct-v03_aggressiveness_score',
 'GEVAL_gpt-4_aggressiveness_score',
  'GEVAL_gpt-4_coherence_score',
  'GEVAL_gpt-4_relevance_score',
  'GEVAL_gpt-4_suitableness_score',
 'GEVAL_Llama-3-8b-chat-hf_aggressiveness_score',
 'GEVAL_Llama-3-8b-chat-hf_coherence_score',
 'GEVAL_Llama-3-8b-chat-hf_relevance_score',
 'GEVAL_Llama-3-8b-chat-hf_suitableness_score',
 'GEVAL_Mistral-7B-Instruct-v03_coherence_score',
 'GEVAL_Mistral-7B-Instruct-v03_relevance_score',
 'GEVAL_Mistral-7B-Instruct-v03_suitableness_score',
 'GEVAL_Mistral-7B-Instruct-v03_aggressiveness_score',
 'Llama-3-8b-chat-hf_relevance_score',
 'Llama-3-8b-chat-hf_coherence_score',
 'Llama-3-8b-chat-hf_suitableness_score',
 'Llama-3-8b-chat-hf_aggressiveness_score',
 'Mistral-7B-Instruct-v03_relevance_score',
 'Mistral-7B-Instruct-v03_coherence_score',
 'Mistral-7B-Instruct-v03_suitableness_score',
 'Mistral-7B-Instruct-v03_aggressiveness_score',
 'gpt-4_relevance_score',
 'gpt-4_aggressiveness_score',
 'gpt-4_coherence_score',
 'gpt-4_suitableness_score'
 ]

human_ratings = ['relevance_score', 'aggressiveness_score', 'coherence_score', 'suitableness_score']

In [398]:
df_anal = pd.read_csv('/home/amey/depository/cs-eval/data/annotations/df_analysis_human_annotation_metrics_calculated.csv')
print(df_anal.shape)
df_anal.head(1)

(305, 59)


,level_0,index,hatespeech,csType,counterspeech,predicted_counterspeech,source,uuid,prediction_mistralai-Mistral-7B-Instruct-v0.3_zeroshot_relevance,prediction_mistralai-Mistral-7B-Instruct-v0.3_zeroshot_coherence,...,"rouge_2_(pred_cs, cs)","meteor_score_(pred_cs, cs)","bert_score_(hs, pred_cs)",toxicity_(pred_cs),"pc_score_(hs, pred_cs)","negative_pc_score_(hs, pred_cs)","cd_score_(hs, pred_cs)",aq_score_(pred_cs),"pd_score(hs, pred_csv)","bert_score_(pred_cs, cs)"
0,0,0,Above all Islam is a religion of conquest.,Denouncing,"No, islam is a religion of 1.6 billion people ...",islam only a religion of per just wrong. peace...,GPS,944ec738-ee5e-40d6-8493-93c1eb8b403e,2.0,2.0,...,0.137931,0.154321,0.871862,0.457112,0.723042,-0.723042,0.35685,0.492228,-0.366192,0.8534


In [399]:
df_anal = df_anal[traditional_metrics]
df_anal = df_anal.reset_index()
print(df_anal.shape)
df_anal.head()

(305, 10)


,index,"bleu_1_(pred_cs, cs)","bleu_2_(pred_cs, cs)","bleu_3_(pred_cs, cs)","bleu_4_(pred_cs, cs)","rouge_1_(pred_cs, cs)","rouge_2_(pred_cs, cs)","rouge_l_(pred_cs, cs)","meteor_score_(pred_cs, cs)","bert_score_(pred_cs, cs)"
0,0,0.062500,0.031888,0.002702,0.001368,0.275862,0.137931,0.275862,0.154321,0.853400
1,1,0.058824,0.028614,0.002603,0.001317,0.193548,0.000000,0.129032,0.092593,0.856686
2,2,0.013889,0.002453,0.000247,0.000124,0.081633,0.037037,0.081633,0.131926,0.830768
3,3,0.041667,0.018386,0.001648,0.000830,0.270270,0.000000,0.108108,0.176471,0.853999
4,4,0.041667,0.018174,0.001631,0.000821,0.216216,0.000000,0.108108,0.147059,0.854176


In [400]:
# for col in traditional_metrics:
#     if col in df_analysis.columns:
#         df_analysis = df_analysis.drop(columns=col)
#     df_analysis.drop(columns='uuid', inplace=True)

df_analysis = pd.concat([df_analysis, df_anal], axis=1)
# pd.concat([df_analysis, df_anal], axis=1).shape
# assert df_analysis.shape[0] == 305
# df_analysis.head()

In [401]:
df_concat = pd.concat([df_analysis, df_anal], axis=1)
df_concat.head()

,level_0,index,hatespeech,csType,counterspeech,predicted_counterspeech,source,uuid,prediction_mistralai-Mistral-7B-Instruct-v0.3_zeroshot_relevance,prediction_mistralai-Mistral-7B-Instruct-v0.3_zeroshot_coherence,...,index,"bleu_1_(pred_cs, cs)","bleu_2_(pred_cs, cs)","bleu_3_(pred_cs, cs)","bleu_4_(pred_cs, cs)","rouge_1_(pred_cs, cs)","rouge_2_(pred_cs, cs)","rouge_l_(pred_cs, cs)","meteor_score_(pred_cs, cs)","bert_score_(pred_cs, cs)"
0,0,0,Above all Islam is a religion of conquest.,Denouncing,"No, islam is a religion of 1.6 billion people ...",islam only a religion of per just wrong. peace...,GPS,944ec738-ee5e-40d6-8493-93c1eb8b403e,2.0,2.0,...,0,0.062500,0.031888,0.002702,0.001368,0.275862,0.137931,0.275862,0.154321,0.853400
1,1,1,Above all Islam is a religion of conquest.,Denouncing,"No, islam is a religion of 1.6 billion people ...",Is it fair to judge an entire religious group ...,QUARC,b0e53c71-0567-4052-971c-d777b9fcdc1a,4.0,5.0,...,1,0.058824,0.028614,0.002603,0.001317,0.193548,0.000000,0.129032,0.092593,0.856686
2,2,2,Above all Islam is a religion of conquest.,Denouncing,"No, islam is a religion of 1.6 billion people ...",I must strongly denounce your offensive statem...,GPT3.5_ZS,b0449292-e5b5-4e0a-b36e-8eaabc0d3915,5.0,5.0,...,2,0.013889,0.002453,0.000247,0.000124,0.081633,0.037037,0.081633,0.131926,0.830768
3,3,3,Above all Islam is a religion of conquest.,Denouncing,"No, islam is a religion of 1.6 billion people ...",Generalizing about Muslims based on the action...,GPT3.5_FS,NaN,4.0,5.0,...,3,0.041667,0.018386,0.001648,0.000830,0.270270,0.000000,0.108108,0.176471,0.853999
4,4,4,Above all Islam is a religion of conquest.,Denouncing,"No, islam is a religion of 1.6 billion people ...",Stereotyping Muslims based on the actions of a...,dialoGPT,NaN,4.0,4.0,...,4,0.041667,0.018174,0.001631,0.000821,0.216216,0.000000,0.108108,0.147059,0.854176


In [337]:
# df_analysis.shape

In [318]:
# df_ranking.columns

### Statistical significance testing

In [60]:
import scipy.stats as stats

# Example t-statistics
t_A = 1.155
t_B = 5.135
t_C = 3.970
df = 3  # degrees of freedom

# Calculate p-values (two-tailed)
p_A = 2 * (1 - stats.t.cdf(abs(t_A), df))
p_B = 2 * (1 - stats.t.cdf(abs(t_B), df))
p_C = 2 * (1 - stats.t.cdf(abs(t_C), df))

print(f"p-value for Model A: {p_A}")
print(f"p-value for Model B: {p_B}")
print(f"p-value for Model C: {p_C}")

p-value for Model A: 0.33173591402082847
p-value for Model B: 0.014306057386159976
p-value for Model C: 0.02856527352376914


In [61]:
import numpy as np
import pandas as pd
from scipy.stats import spearmanr, t

# Sample data: Ground truth ratings and model ratings
data = {
    'Ground Truth': [4, 3, 5, 2, 1],
    'Model A': [5, 3, 4, 1, 2],
    'Model B': [4, 2, 5, 2, 1],
    'Model C': [3, 4, 5, 2, 1]
}

# Convert to DataFrame
df = pd.DataFrame(data)

# Function to compute Spearman correlation, t-statistic, and p-value
def compute_statistics(df, model_col):
    # Calculate Spearman correlation
    rho, _ = spearmanr(df['Ground Truth'], df[model_col])
    
    # Number of observations
    n = len(df)
    
    # Calculate t-statistic
    t_statistic = (rho * np.sqrt(n - 2)) / np.sqrt(1 - rho**2)
    
    # Degrees of freedom
    df = n - 2
    
    # Calculate p-value (two-tailed)
    p_value = 2 * (1 - t.cdf(np.abs(t_statistic), df))
    
    return rho, t_statistic, p_value

# Results storage
results = []

# Compute statistics for each model
for model in df.columns[1:]:
    rho, t_stat, p_val = compute_statistics(df, model)
    results.append({
        'Model': model,
        'Spearman Correlation (ρ)': rho,
        't-statistic': t_stat,
        'p-value': p_val
    })

# Convert results to DataFrame for better visualization
results_df = pd.DataFrame(results)

# Display results
print(results_df)

     Model  Spearman Correlation (ρ)  t-statistic   p-value
0  Model A                  0.800000     2.309401  0.104088
1  Model B                  0.974679     7.549834  0.004818
2  Model C                  0.900000     3.576237  0.037386


In [62]:
df.head()

,Ground Truth,Model A,Model B,Model C
0,4,5,4,3
1,3,3,2,4
2,5,4,5,5
3,2,1,2,2
4,1,2,1,1


In [66]:
df_results = pd.read_csv('/home/amey/depository/cs-eval/emnlp_2024_analysis/analysis_metrics_computed_renamed_cols.csv')
df_results = df_results.sample(5000)
df_results.head(1)

,index,hatespeech,counterspeech,predicted_counterspeech,csType,suitableness_score,relevance_score,coherence_score,aggressiveness_score,annotator,...,GEVAL_Llama-3-8b-chat-hf_aggressiveness_score,GEVAL_gpt-4_coherence_score,GEVAL_Llama-3-8b-chat-hf_coherence_score,GEVAL_Mistral-7B-Instruct-v03_coherence_score,GEVAL_gpt-4_relevance_score,GEVAL_Llama-3-8b-chat-hf_relevance_score,GEVAL_Mistral-7B-Instruct-v03_relevance_score,GEVAL_gpt-4_suitableness_score,GEVAL_Mistral-7B-Instruct-v03_suitableness_score,GEVAL_Llama-3-8b-chat-hf_suitableness_score
1540,NaN,Islam is a destructive force in the world.,Can we generalize about an entire religion? is...,How can you generalize an entire religion base...,Questioning,2.0,5.0,3.0,1.0,NaN,...,1.0,2.0,2.0,4.0,5.0,4.0,5.0,2.0,3.0,2.0


In [67]:
for col in df_results.columns:
    print(col)

index
hatespeech
counterspeech
predicted_counterspeech
csType
suitableness_score
relevance_score
coherence_score
aggressiveness_score
annotator
source
uuid
id
bart_score_(cs, pred_cs)
bart_score_(hs, pred_cs)
bart_score_(pred_cs, cs)
coherence_UniEval
relevance_UniEval
aggressiveness_UniEval
suitableness_UniEval
bm25_score_(pred_cs, cs)
bm25_score_(hs, pred_cs)
mauve_score_(hs, pred_cs)
mauve_score_(pred_cs, cs)
bleu_1_(pred_cs, cs)
bleu_2_(pred_cs, cs)
bleu_3_(pred_cs, cs)
bleu_4_(pred_cs, cs)
rouge_1_(pred_cs, cs)
rouge_2_(pred_cs, cs)
rouge_l_(pred_cs, cs)
meteor_score_(pred_cs, cs)
bert_score_(hs, pred_cs)
bert_score_(pred_cs, cs)
pc_score_(hs, pred_cs)
aq_score_(pred_cs)
cd_score_(hs, pred_cs)
pd_score(hs, pred_cs)
negative_pc_score_(hs, pred_cs)
toxicity_(pred_cs)
obscenity_(pred_cs)
identity_attack_(pred_cs)
insult_(pred_cs)
gpt-4_relevance_score
gpt-4_aggressiveness_score
gpt-4_coherence_score
gpt-4_suitableness_score
gpt-4_suitablness_score
gpt-4-zs_relevance_score
gpt-4-zs_co

In [68]:
traditional_metrics = [
'bleu_1_(pred_cs, cs)',
'bleu_2_(pred_cs, cs)',
'bleu_3_(pred_cs, cs)',
'bleu_4_(pred_cs, cs)',
'rouge_1_(pred_cs, cs)',
'rouge_2_(pred_cs, cs)',
'rouge_l_(pred_cs, cs)',
'meteor_score_(pred_cs, cs)',
'bert_score_(pred_cs, cs)',
'pc_score_(hs, pred_cs)',
'aq_score_(pred_cs)',
'pd_score(hs, pred_cs)',
'negative_pc_score_(hs, pred_cs)',
'toxicity_(pred_cs)',
'bart_score_(pred_cs, cs)',
]

unieval_cols = [
'aggressiveness_UniEval',
'coherence_UniEval',
'relevance_UniEval',
'suitableness_UniEval',
]

prediction_cols = [
'gpt-4-zs_relevance_score',
'gpt-4-zs_coherence_score',
'gpt-4-zs_aggressiveness_score',
'gpt-4-zs_suitableness_score',
'zs_Llama-3-8b-chat-hf_relevance_score',
'zs_Llama-3-8b-chat-hf_coherence_score',
'zs_Llama-3-8b-chat-hf_suitableness_score',
'zs_Llama-3-8b-chat-hf_aggressiveness_score',
'zs_Mistral-7B-Instruct-v03_relevance_score',
'zs_Mistral-7B-Instruct-v03_coherence_score',
'zs_Mistral-7B-Instruct-v03_suitableness_score',
'zs_Mistral-7B-Instruct-v03_aggressiveness_score',
'GEVAL_gpt-4_aggressiveness_score',
'GEVAL_gpt-4_coherence_score',
'GEVAL_gpt-4_relevance_score',
'GEVAL_gpt-4_suitableness_score',
'GEVAL_Llama-3-8b-chat-hf_aggressiveness_score',
'GEVAL_Llama-3-8b-chat-hf_coherence_score',
'GEVAL_Llama-3-8b-chat-hf_relevance_score',
'GEVAL_Llama-3-8b-chat-hf_suitableness_score',
'GEVAL_Mistral-7B-Instruct-v03_coherence_score',
'GEVAL_Mistral-7B-Instruct-v03_relevance_score',
'GEVAL_Mistral-7B-Instruct-v03_suitableness_score',
'GEVAL_Mistral-7B-Instruct-v03_aggressiveness_score',
'Llama-3-8b-chat-hf_relevance_score',
'Llama-3-8b-chat-hf_coherence_score',
'Llama-3-8b-chat-hf_suitableness_score',
'Llama-3-8b-chat-hf_aggressiveness_score',
'Mistral-7B-Instruct-v03_relevance_score',
'Mistral-7B-Instruct-v03_coherence_score',
'Mistral-7B-Instruct-v03_suitableness_score',
'Mistral-7B-Instruct-v03_aggressiveness_score',
'gpt-4_relevance_score',
'gpt-4_aggressiveness_score',
'gpt-4_coherence_score',
'gpt-4_suitableness_score'
]

human_ratings = ['relevance_score', 'aggressiveness_score', 'coherence_score', 'suitableness_score']

In [69]:
# Compute SS for Relevance
aspect_name = 'coherence_score'
y_pred_cols = [col for col in prediction_cols if aspect_name in col or f'{aspect_name}_UniEval' in col] + ['coherence_UniEval']
y_pred_cols += traditional_metrics
y_true_cols = [col for col in human_ratings if aspect_name in col]
total_cols = y_true_cols + y_pred_cols
total_cols

['coherence_score',
 'gpt-4-zs_coherence_score',
 'zs_Llama-3-8b-chat-hf_coherence_score',
 'zs_Mistral-7B-Instruct-v03_coherence_score',
 'GEVAL_gpt-4_coherence_score',
 'GEVAL_Llama-3-8b-chat-hf_coherence_score',
 'GEVAL_Mistral-7B-Instruct-v03_coherence_score',
 'Llama-3-8b-chat-hf_coherence_score',
 'Mistral-7B-Instruct-v03_coherence_score',
 'gpt-4_coherence_score',
 'coherence_UniEval',
 'bleu_1_(pred_cs, cs)',
 'bleu_2_(pred_cs, cs)',
 'bleu_3_(pred_cs, cs)',
 'bleu_4_(pred_cs, cs)',
 'rouge_1_(pred_cs, cs)',
 'rouge_2_(pred_cs, cs)',
 'rouge_l_(pred_cs, cs)',
 'meteor_score_(pred_cs, cs)',
 'bert_score_(pred_cs, cs)',
 'pc_score_(hs, pred_cs)',
 'aq_score_(pred_cs)',
 'pd_score(hs, pred_cs)',
 'negative_pc_score_(hs, pred_cs)',
 'toxicity_(pred_cs)',
 'bart_score_(pred_cs, cs)']

In [70]:
df_view = df_results[total_cols]
df_view.head()

,coherence_score,gpt-4-zs_coherence_score,zs_Llama-3-8b-chat-hf_coherence_score,zs_Mistral-7B-Instruct-v03_coherence_score,GEVAL_gpt-4_coherence_score,GEVAL_Llama-3-8b-chat-hf_coherence_score,GEVAL_Mistral-7B-Instruct-v03_coherence_score,Llama-3-8b-chat-hf_coherence_score,Mistral-7B-Instruct-v03_coherence_score,gpt-4_coherence_score,...,"rouge_2_(pred_cs, cs)","rouge_l_(pred_cs, cs)","meteor_score_(pred_cs, cs)","bert_score_(pred_cs, cs)","pc_score_(hs, pred_cs)",aq_score_(pred_cs),"pd_score(hs, pred_cs)","negative_pc_score_(hs, pred_cs)",toxicity_(pred_cs),"bart_score_(pred_cs, cs)"
1540,3.0,3.0,3.0,4.0,2.0,2.0,4.0,2.0,4.0,2.0,...,0.062500,0.294118,0.279243,0.894743,-0.816182,0.766144,1.755184,0.816182,0.000664,-3.339529
2806,3.0,5.0,4.0,4.0,2.0,4.0,4.0,4.0,4.0,2.0,...,0.051502,0.178344,0.259558,0.855028,0.999399,0.924829,-0.003144,-0.999399,0.004222,-3.288932
4464,3.0,4.0,4.0,4.0,2.0,2.0,4.0,2.0,4.0,2.0,...,0.000000,0.058824,0.070755,0.858426,0.998735,0.717073,-0.065075,-0.998735,0.120347,-3.895826
4783,1.0,1.0,2.0,2.0,2.0,2.0,3.0,2.0,3.0,1.0,...,0.000000,0.181818,0.108303,0.842483,0.998498,0.812375,-0.875203,-0.998498,0.011965,-3.423391
3492,3.0,3.0,3.0,4.0,4.0,3.0,5.0,3.0,5.0,3.0,...,0.025974,0.173913,0.096875,0.844502,-0.921634,0.856986,1.532287,0.921634,0.089539,-2.931246


In [71]:
# Function to compute Spearman correlation, t-statistic, and p-value
def compute_statistics(df, model_col, y_true_col):
    # Calculate Spearman correlation
    rho, _ = spearmanr(df[y_true_col], df[model_col])
    
    # Number of observations
    n = len(df)
    
    # Calculate t-statistic
    t_statistic = (rho * np.sqrt(n - 2)) / np.sqrt(1 - rho**2)
    
    # Degrees of freedom
    df = n - 2
    
    # Calculate p-value (two-tailed)
    p_value = 2 * (1 - t.cdf(np.abs(t_statistic), df))
    
    return rho, t_statistic, p_value

# Results storage
results = []

# Compute statistics for each model
for model in df_view.columns[1:]:
    rho, t_stat, p_val = compute_statistics(df_view, model, y_true_col="coherence_score")
    results.append({
        'Model': model,
        'Spearman Correlation (ρ)': rho,
        't-statistic': t_stat,
        'p-value': p_val
    })

# Convert results to DataFrame for better visualization
statistical_test_df = pd.DataFrame(results)

# Display results
# print(statistical_test_df)
statistical_test_df.head(100)

,Model,Spearman Correlation (ρ),t-statistic,p-value
0,gpt-4-zs_coherence_score,0.439583,34.599106,0.000000e+00
1,zs_Llama-3-8b-chat-hf_coherence_score,0.336586,25.269910,0.000000e+00
2,zs_Mistral-7B-Instruct-v03_coherence_score,0.377003,28.776113,0.000000e+00
3,GEVAL_gpt-4_coherence_score,0.583905,50.848662,0.000000e+00
4,GEVAL_Llama-3-8b-chat-hf_coherence_score,0.322556,24.091236,0.000000e+00
5,GEVAL_Mistral-7B-Instruct-v03_coherence_score,0.346971,26.154451,0.000000e+00
6,Llama-3-8b-chat-hf_coherence_score,0.330840,24.784963,0.000000e+00
7,Mistral-7B-Instruct-v03_coherence_score,0.355565,26.894710,0.000000e+00
8,gpt-4_coherence_score,0.601487,53.228117,0.000000e+00
9,coherence_UniEval,0.169098,12.129294,0.000000e+00


In [72]:
statistical_test_df['p-value'].round(8)

0     0.000000
1     0.000000
2     0.000000
3     0.000000
4     0.000000
5     0.000000
6     0.000000
7     0.000000
8     0.000000
9     0.000000
10    0.000000
11    0.000000
12    0.000000
13    0.000000
14    0.000000
15    0.000000
16    0.000000
17    0.000000
18    0.000000
19    0.000000
20    0.001479
21    0.000000
22    0.000000
23    0.000000
24    0.000000
Name: p-value, dtype: float64

In [73]:
statistical_test_df.head()
statistical_test_df = statistical_test_df.sort_values(by='Model')
statistical_test_df

,Model,Spearman Correlation (ρ),t-statistic,p-value
4,GEVAL_Llama-3-8b-chat-hf_coherence_score,0.322556,24.091236,0.000000e+00
5,GEVAL_Mistral-7B-Instruct-v03_coherence_score,0.346971,26.154451,0.000000e+00
3,GEVAL_gpt-4_coherence_score,0.583905,50.848662,0.000000e+00
6,Llama-3-8b-chat-hf_coherence_score,0.330840,24.784963,0.000000e+00
7,Mistral-7B-Instruct-v03_coherence_score,0.355565,26.894710,0.000000e+00
20,aq_score_(pred_cs),0.044943,3.180507,1.479162e-03
24,"bart_score_(pred_cs, cs)",0.383519,29.358405,0.000000e+00
18,"bert_score_(pred_cs, cs)",0.342531,25.774948,0.000000e+00
10,"bleu_1_(pred_cs, cs)",0.102512,7.285658,3.701484e-13
11,"bleu_2_(pred_cs, cs)",-0.232870,-16.928471,0.000000e+00


In [231]:
statistical_test_df.to_csv('/home/amey/depository/cs-eval/statistical_tests/coherence_complete.csv')

In [176]:
# results_dict = {}

results_dict['coherence'] = statistical_test_df

In [177]:
result_aggregted = pd.DataFrame()

for key in results_dict:
    df = results_dict[key]
    if result_aggregted.shape[0] == 0:
        result_aggregted = result_aggregted.merge(df)
    else:
        result_aggregted = result_aggregted.merge(df, on=["Model"],suffixes=[f'_{key}',f'_{key}'])

result_aggregted

,Model,Spearman Correlation (ρ)_coherence,t-statistic_coherence,p-value_coherence,Spearman Correlation (ρ)_coherence,t-statistic_coherence,p-value_coherence
0,aq_score_(pred_cs),0.273408,8.979414,0.000000e+00,0.169892,5.446264,6.477786e-08
1,"bart_score_(pred_cs, cs)",0.253230,8.269349,4.440892e-16,0.259169,8.477075,0.000000e+00
2,"bert_score_(pred_cs, cs)",0.075047,2.377528,1.761669e-02,0.066599,2.108631,3.522511e-02
3,"bleu_1_(pred_cs, cs)",0.022391,0.707539,4.793966e-01,-0.014509,-0.458419,6.467514e-01
4,"bleu_2_(pred_cs, cs)",-0.316866,-10.553986,0.000000e+00,-0.437443,-15.367679,0.000000e+00
5,"bleu_3_(pred_cs, cs)",-0.322465,-10.761921,0.000000e+00,-0.447044,-15.788084,0.000000e+00
6,"bleu_4_(pred_cs, cs)",-0.322465,-10.761921,0.000000e+00,-0.447044,-15.788084,0.000000e+00
7,"meteor_score_(pred_cs, cs)",0.257354,8.413506,0.000000e+00,0.285534,9.412179,0.000000e+00
8,"negative_pc_score_(hs, pred_cs)",0.155645,4.977654,7.578055e-07,0.096535,3.063976,2.242796e-03
9,"pc_score_(hs, pred_cs)",-0.155645,-4.977654,7.578055e-07,-0.096535,-3.063976,2.242796e-03


In [125]:
keep_cols = ["Model"] + [col for col in result_aggregted.columns if 'p-' in col]
result_aggregted

,Model,Spearman Correlation (ρ)_aggressiveness,t-statistic_aggressiveness,p-value_aggressiveness,Spearman Correlation (ρ)_aggressiveness,t-statistic_aggressiveness,p-value_aggressiveness,Spearman Correlation (ρ)_suitableness,t-statistic_suitableness,p-value_suitableness,Spearman Correlation (ρ)_suitableness,t-statistic_suitableness,p-value_suitableness
0,aq_score_(pred_cs),0.139648,11.391146,0.000000e+00,0.014866,1.200861,2.298489e-01,0.051013,4.125790,0.000037,0.051901,4.197743,2.731700e-05
1,"bart_score_(pred_cs, cs)",0.276174,23.209640,0.000000e+00,-0.196778,-16.210928,0.000000e+00,0.388745,34.079955,0.000000,0.255486,21.344271,0.000000e+00
2,"bert_score_(pred_cs, cs)",0.222516,18.435056,0.000000e+00,-0.074823,-6.060545,1.432281e-09,0.347490,29.932473,0.000000,0.250449,20.894989,0.000000e+00
3,"bleu_1_(pred_cs, cs)",0.001671,0.134975,8.926358e-01,-0.137347,-11.199802,0.000000e+00,0.104392,8.478206,0.000000,0.009767,0.788954,4.301674e-01
4,"bleu_2_(pred_cs, cs)",-0.075129,-6.085463,1.227414e-09,0.155197,12.689240,0.000000e+00,-0.230571,-19.139197,0.000000,-0.097109,-7.880858,3.774758e-15
5,"bleu_3_(pred_cs, cs)",-0.090728,-7.358526,2.087219e-13,0.159741,13.070286,0.000000e+00,-0.251044,-20.947949,0.000000,-0.114336,-9.296003,0.000000e+00
6,"bleu_4_(pred_cs, cs)",-0.090728,-7.358526,2.087219e-13,0.159741,13.070286,0.000000e+00,-0.251044,-20.947949,0.000000,-0.114336,-9.296003,0.000000e+00
7,"meteor_score_(pred_cs, cs)",0.171473,14.058355,0.000000e+00,-0.204731,-16.894225,0.000000e+00,0.352934,30.467594,0.000000,0.166009,13.597465,0.000000e+00
8,"negative_pc_score_(hs, pred_cs)",-0.018940,-1.530103,1.260397e-01,-0.221548,-18.350705,0.000000e+00,0.105847,8.597737,0.000000,0.004704,0.379971,7.039791e-01
9,"pc_score_(hs, pred_cs)",0.018940,1.530103,1.260397e-01,0.221548,18.350705,0.000000e+00,-0.105847,-8.597737,0.000000,-0.004704,-0.379971,7.039791e-01


In [153]:
result_aggregted.to_csv('/home/amey/depository/cs-eval/statistical_tests/top_100_results.csv')

In [156]:
df = pd.read_csv('/home/amey/depository/cs-eval/statistical_tests/complete_results.csv')
df.head()

,Unnamed: 0,Model,Spearman Correlation (ρ)_aggressiveness,t-statistic_aggressiveness,p-value_aggressiveness,Spearman Correlation (ρ)_aggressiveness.1,t-statistic_aggressiveness.1,p-value_aggressiveness.1,Spearman Correlation (ρ)_suitableness,t-statistic_suitableness,p-value_suitableness,Spearman Correlation (ρ)_suitableness.1,t-statistic_suitableness.1,p-value_suitableness.1
0,0,aq_score_(pred_cs),0.139648,11.391146,0.000000e+00,0.014866,1.200861,2.298489e-01,0.051013,4.125790,0.000037,0.051901,4.197743,2.731700e-05
1,1,"bart_score_(pred_cs, cs)",0.276174,23.209640,0.000000e+00,-0.196778,-16.210928,0.000000e+00,0.388745,34.079955,0.000000,0.255486,21.344271,0.000000e+00
2,2,"bert_score_(pred_cs, cs)",0.222516,18.435056,0.000000e+00,-0.074823,-6.060545,1.432281e-09,0.347490,29.932473,0.000000,0.250449,20.894989,0.000000e+00
3,3,"bleu_1_(pred_cs, cs)",0.001671,0.134975,8.926358e-01,-0.137347,-11.199802,0.000000e+00,0.104392,8.478206,0.000000,0.009767,0.788954,4.301674e-01
4,4,"bleu_2_(pred_cs, cs)",-0.075129,-6.085463,1.227414e-09,0.155197,12.689240,0.000000e+00,-0.230571,-19.139197,0.000000,-0.097109,-7.880858,3.774758e-15


#### paired t-tests

In [32]:
df = pd.read_csv('/home/amey/depository/cs-eval/statistical_tests/aggressiveness_complete.csv')
df = df[['Model', 'Spearman Correlation (ρ)']]
df = df.transpose()
# df = df.reset_index(drop=True)
df.head(5)
df.columns


RangeIndex(start=0, stop=25, step=1)

In [17]:
from scipy.stats import shapiro
columns = df.columns.tolist()
for model in columns:
    results = df_view[model]
    _, p_value = shapiro(results)
    if p_value < 0.05:
        print(f"Model {model} results are not normally distributed (p={p_value:.3f})")
    else:
        print(f"Model {model} results are normally distributed (p={p_value:.3f})")

Model coherence_score results are not normally distributed (p=0.000)
Model gpt-4-zs_coherence_score results are not normally distributed (p=0.000)
Model zs_Llama-3-8b-chat-hf_coherence_score results are not normally distributed (p=0.000)
Model zs_Mistral-7B-Instruct-v03_coherence_score results are not normally distributed (p=0.000)
Model GEVAL_gpt-4_coherence_score results are not normally distributed (p=0.000)
Model GEVAL_Llama-3-8b-chat-hf_coherence_score results are not normally distributed (p=0.000)
Model GEVAL_Mistral-7B-Instruct-v03_coherence_score results are not normally distributed (p=0.000)
Model Llama-3-8b-chat-hf_coherence_score results are not normally distributed (p=0.000)
Model Mistral-7B-Instruct-v03_coherence_score results are not normally distributed (p=0.000)
Model gpt-4_coherence_score results are not normally distributed (p=0.000)
Model coherence_UniEval results are not normally distributed (p=0.000)
Model bleu_1_(pred_cs, cs) results are not normally distributed (

In [38]:
df1 = pd.read_csv('/home/amey/depository/cs-eval/results/spearman_aggressiveness_score.csv')
df2 = pd.read_csv('/home/amey/depository/cs-eval/results/kendalltau_aggressiveness_score.csv')
df = pd.merge(df1, df2, on='metric', suffixes=['spearman', 'kendalltau'])
assert df.shape[0] == df1.shape[0] == df2.shape[0]
df.head()

,aggressiveness_scorespearman,metric,aggressiveness_scorekendalltau
0,-0.137347,"bleu_1_(pred_cs, cs)",-0.113858
1,0.155197,"bleu_2_(pred_cs, cs)",0.119664
2,0.159741,"bleu_3_(pred_cs, cs)",0.123370
3,0.159741,"bleu_4_(pred_cs, cs)",0.123370
4,-0.166764,"rouge_1_(pred_cs, cs)",-0.130036


In [43]:
from scipy.stats import wilcoxon
from itertools import combinations
model_names = df.metric.values.tolist()
for metric in ['aggressiveness_scorespearman', 'aggressiveness_scorekendalltau']:
    for model1, model2 in combinations(model_names, 2):
        print(model1, model2)
        model1_values = df.loc[df['metric'] == model1, metric].values
        model2_values = df.loc[df['metric'] == model2, metric].values
        t_stat, p_value = wilcoxon(model1_values, model2_values)
        print(f"Comparing Model {model1} and Model {model2} using {metric.capitalize()} correlation:")
        print(f"t-statistic={t_stat:.3f}, p-value={p_value:.3f}")

bleu_1_(pred_cs, cs) bleu_2_(pred_cs, cs)
Comparing Model bleu_1_(pred_cs, cs) and Model bleu_2_(pred_cs, cs) using Aggressiveness_scorespearman correlation:
t-statistic=0.000, p-value=1.000
bleu_1_(pred_cs, cs) bleu_3_(pred_cs, cs)
Comparing Model bleu_1_(pred_cs, cs) and Model bleu_3_(pred_cs, cs) using Aggressiveness_scorespearman correlation:
t-statistic=0.000, p-value=1.000
bleu_1_(pred_cs, cs) bleu_4_(pred_cs, cs)
Comparing Model bleu_1_(pred_cs, cs) and Model bleu_4_(pred_cs, cs) using Aggressiveness_scorespearman correlation:
t-statistic=0.000, p-value=1.000
bleu_1_(pred_cs, cs) rouge_1_(pred_cs, cs)
Comparing Model bleu_1_(pred_cs, cs) and Model rouge_1_(pred_cs, cs) using Aggressiveness_scorespearman correlation:
t-statistic=0.000, p-value=1.000
bleu_1_(pred_cs, cs) rouge_2_(pred_cs, cs)
Comparing Model bleu_1_(pred_cs, cs) and Model rouge_2_(pred_cs, cs) using Aggressiveness_scorespearman correlation:
t-statistic=0.000, p-value=1.000
bleu_1_(pred_cs, cs) rouge_l_(pred_cs, cs

ValueError: zero_method 'wilcox' and 'pratt' do not work if x - y is zero for all elements.

### Statistical significance testing - across different temperature values

In [23]:
import glob
from utils import postprocess_llm_response, extract_content_from_response

df1 = pd.read_pickle('/home/amey/depository/cs-eval/predictions_rebuttal/mistralai-Mistral-7B-Instruct-v0.3_temperature_0.0.pkl')
df2 = pd.read_pickle('/home/amey/depository/cs-eval/predictions_rebuttal/mistralai-Mistral-7B-Instruct-v0.3_temperature_0.5.pkl')
df3 = pd.read_pickle('/home/amey/depository/cs-eval/predictions_rebuttal/mistralai-Mistral-7B-Instruct-v0.3_temperature_1.pkl')

prediction_cols = [col for col in df1.columns if 'prediction_' in col]

for col in prediction_cols:
    df1[col] = df1[col].apply(lambda x: extract_content_from_response(x))
    df1[col] = df1[col].apply(lambda x: extract_score(x))
    df1 = df1.rename(columns={col: col.replace('mistralai/Mistral-7B-Instruct-v0.3_','')})

    df2[col] = df2[col].apply(lambda x: extract_content_from_response(x))
    df2[col] = df2[col].apply(lambda x: extract_score(x))
    df2 = df2.rename(columns={col: col.replace('mistralai/Mistral-7B-Instruct-v0.3_','')})

    df3[col] = df3[col].apply(lambda x: extract_content_from_response(x))
    df3[col] = df3[col].apply(lambda x: extract_score(x))
    df3 = df3.rename(columns={col: col.replace('mistralai/Mistral-7B-Instruct-v0.3_','')})

prediction_cols = [col for col in df1.columns if 'prediction_' in col] 
# df_analysis = pd.concat([df1[prediction_cols], df2[prediction_cols], df3[prediction_cols]], axis=1)
df_analysis = pd.concat([df1[prediction_cols], df2[prediction_cols], df3[prediction_cols]], axis=1)
df_analysis = rename_similar_columns(df_analysis)
df_analysis.head()

,prediction_relevance_score,prediction_coherence_score,prediction_suitableness_score,prediction_aggressiveness_score,prediction_relevance_score,prediction_coherence_score,prediction_suitableness_score,prediction_aggressiveness_score,prediction_relevance_score,prediction_coherence_score,prediction_suitableness_score,prediction_aggressiveness_score
0,4.0,3.0,3.0,2.0,4.0,3.0,3.0,3.0,4.0,2.0,3.0,2.0
1,4.0,4.0,3.0,2.0,4.0,4.0,3.0,2.0,4.0,4.0,3.0,2.0
2,5.0,5.0,3.0,2.0,5.0,5.0,3.0,2.0,5.0,5.0,3.0,2.0
3,1.0,1.0,1.0,4.0,1.0,1.0,1.0,4.0,1.0,1.0,1.0,4.0
4,4.0,2.0,2.0,2.0,5.0,2.0,2.0,3.0,4.0,2.0,2.0,2.0


In [24]:
df_w_gt = pd.read_csv('/home/amey/depository/cs-eval/data/annotations/dataset_analysis_metrics_calculated.csv')[:1300]
true_cols = [col.replace('prediction_','') for col in prediction_cols]
df_analysis = pd.concat([df_analysis, df_w_gt[true_cols]], axis=1)
df_analysis.head()

,prediction_relevance_score,prediction_coherence_score,prediction_suitableness_score,prediction_aggressiveness_score,prediction_relevance_score,prediction_coherence_score,prediction_suitableness_score,prediction_aggressiveness_score,prediction_relevance_score,prediction_coherence_score,prediction_suitableness_score,prediction_aggressiveness_score,relevance_score,coherence_score,suitableness_score,aggressiveness_score
0,4.0,3.0,3.0,2.0,4.0,3.0,3.0,3.0,4.0,2.0,3.0,2.0,3.333333,2.333333,1.666667,1.666667
1,4.0,4.0,3.0,2.0,4.0,4.0,3.0,2.0,4.0,4.0,3.0,2.0,3.666667,4.333333,2.333333,1.000000
2,5.0,5.0,3.0,2.0,5.0,5.0,3.0,2.0,5.0,5.0,3.0,2.0,3.333333,3.333333,2.666667,1.000000
3,1.0,1.0,1.0,4.0,1.0,1.0,1.0,4.0,1.0,1.0,1.0,4.0,3.000000,1.000000,1.000000,2.333333
4,4.0,2.0,2.0,2.0,5.0,2.0,2.0,3.0,4.0,2.0,2.0,2.0,2.333333,1.000000,1.000000,1.666667


In [25]:
df_analysis['prediction_relevance_score']

,prediction_relevance_score,prediction_relevance_score,prediction_relevance_score
0,4.0,4.0,4.0
1,4.0,4.0,4.0
2,5.0,5.0,5.0
3,1.0,1.0,1.0
4,4.0,5.0,4.0
...,...,...,...
1295,5.0,5.0,5.0
1296,4.0,4.0,4.0
1297,5.0,5.0,5.0
1298,5.0,5.0,5.0


In [27]:
for col in prediction_cols:
    print(col)
    print(f"Standard deviation: {df_analysis[col].std(axis=1).mean()}")

for col in prediction_cols:
    print(col)
    values = df_analysis[col].values.tolist()
    values1 = [x[0] for x in values]
    values2 = [x[1] for x in values]
    values3 = [x[2] for x in values]

    h_statistic, p_value = stats.kruskal(values1, values2, values3)
    print(f"Kruskal-Wallis H-statistic: {h_statistic}, p-value: {p_value}")

prediction_relevance_score
Standard deviation: 0.1288423089857736
prediction_coherence_score
Standard deviation: 0.1480647207294224
prediction_suitableness_score
Standard deviation: 0.034641016151377546
prediction_aggressiveness_score
Standard deviation: 0.11446282224041861
prediction_relevance_score
Kruskal-Wallis H-statistic: 1.1450653426398427, p-value: 0.5640949608699841
prediction_coherence_score
Kruskal-Wallis H-statistic: 0.08925124150100705, p-value: 0.9563554544574492
prediction_suitableness_score
Kruskal-Wallis H-statistic: 0.052638327644834566, p-value: 0.9740241667175431
prediction_aggressiveness_score
Kruskal-Wallis H-statistic: 2.8838521607090604, p-value: 0.2364718559742476


In [21]:
import glob
from utils import postprocess_llm_response, extract_content_from_response

df1 = pd.read_pickle('/home/amey/depository/cs-eval/predictions_rebuttal/meta-llama-Llama-3-8b-chat-hf_temperature_0.pkl')
df2 = pd.read_pickle('/home/amey/depository/cs-eval/predictions_rebuttal/meta-llama-Llama-3-8b-chat-hf_temperature_0.5.pkl')
df3 = pd.read_pickle('/home/amey/depository/cs-eval/predictions_rebuttal/meta-llama-Llama-3-8b-chat-hf_temperature_1.pkl')

prediction_cols = [col for col in df1.columns if 'prediction_' in col]

for col in prediction_cols:
    df1[col] = df1[col].apply(lambda x: extract_content_from_response(x))
    df1[col] = df1[col].apply(lambda x: extract_score(x))
    df1 = df1.rename(columns={col: col.replace('meta-llama/Llama-3-8b-chat-hf_','temp0_')})

    df2[col] = df2[col].apply(lambda x: extract_content_from_response(x))
    df2[col] = df2[col].apply(lambda x: extract_score(x))
    df2 = df2.rename(columns={col: col.replace('meta-llama/Llama-3-8b-chat-hf_','temp0.5_')})

    df3[col] = df3[col].apply(lambda x: extract_content_from_response(x))
    df3[col] = df3[col].apply(lambda x: extract_score(x))
    df3 = df3.rename(columns={col: col.replace('meta-llama/Llama-3-8b-chat-hf_','temp1_')})

df_analysis = pd.concat([df1[[col for col in df1.columns if 'prediction_' in col]], df2[[col for col in df2.columns if 'prediction_' in col]], df3[[col for col in df3.columns if 'prediction_' in col]]], axis=1)
prediction_cols = [col for col in df_analysis.columns if 'prediction_' in col] 
df_analysis.head()

# df_w_gt = pd.read_csv('/home/amey/depository/cs-eval/data/annotations/dataset_analysis_metrics_calculated.csv')[:1300]
# true_cols = ['relevance_score', 'coherence_score', 'aggressiveness_score', 'suitableness_score']
# df_analysis = pd.concat([df_analysis, df_w_gt[true_cols]], axis=1)
# df_analysis.head()

,prediction_temp0_relevance_score,prediction_temp0_coherence_score,prediction_temp0_suitableness_score,prediction_temp0_aggressiveness_score,prediction_temp0.5_relevance_score,prediction_temp0.5_coherence_score,prediction_temp0.5_suitableness_score,prediction_temp0.5_aggressiveness_score,prediction_temp1_relevance_score,prediction_temp1_coherence_score,prediction_temp1_suitableness_score,prediction_temp1_aggressiveness_score
0,5.0,4.0,2.0,1.0,5.0,5.0,2.0,1.0,5.0,5.0,2.0,1.0
1,5.0,4.0,2.0,1.0,4.0,4.0,2.0,1.0,5.0,4.0,2.0,2.0
2,5.0,4.0,2.0,1.0,5.0,4.0,2.0,1.0,5.0,4.0,2.0,1.0
3,1.0,1.0,1.0,4.0,1.0,1.0,1.0,4.0,1.0,1.0,1.0,5.0
4,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0


In [62]:
aspect_name = 'suitableness_score'
y_pred_cols = [col for col in df_analysis.columns if aspect_name in col and 'prediction' in col]
y_true_cols = [aspect_name]

spearman_df, kendall_df = calculate_correlation_matrices(df_analysis, y_pred_cols, y_true_cols)
spearman_df
# spearman_df = spearman_df.rename_axis('metric')
# spearman_df['metric'] = spearman_df.index
# spearman_df.head(500)

,suitableness_score
prediction_temp0_suitableness_score,0.414468
prediction_temp0.5_suitableness_score,0.373575
prediction_temp1_suitableness_score,0.319541


In [46]:
# for col in prediction_cols:
#     print(col)
#     print(f"Standard deviation: {df_analysis[col].std(axis=1).mean()}")

In [ ]:
# import pandas as pd
# from scipy.stats import ttest_rel

# # Load your dataframe
# df = pd.read_csv('your_dataframe.csv')

# # Define the models
# models = ['Model A', 'Model B', 'Model C']

# # Create an empty list to store the results
# results = []

# # Perform paired t-test for each model
# for model in models:
#     t_stat, p_value = ttest_rel(df['y_true'], df[f"{model}_pred"])
#     results.append({
#         'Model': model,
#         't-statistic': t_stat,
#         'p-value': p_value,
#         'Significant Difference': 'Yes' if p_value < 0.05 else 'No'
#     })

# # Create a dataframe from the results
# results_df = pd.DataFrame(results)

# # Display the results table
# print(results_df)

In [25]:
df_analysis['prediction_relevance_score']
# df_anal = df_analysis['prediction_relevance_score']
# spearman_df, kendall_df = calculate_correlation_matrices(, unieval_cols, human_ratings)


,prediction_relevance_score,prediction_relevance_score,prediction_relevance_score
0,5.0,5.0,5.0
1,5.0,4.0,5.0
2,5.0,5.0,5.0
3,1.0,1.0,1.0
4,2.0,1.0,1.0
...,...,...,...
1295,5.0,5.0,5.0
1296,4.0,5.0,4.0
1297,5.0,5.0,5.0
1298,4.0,4.0,4.0


In [102]:
import glob
from utils import postprocess_llm_response, extract_content_from_response

df1 = pd.read_pickle('/home/amey/depository/cs-eval/predictions_rebuttal/meta-llama-Llama-3-8b-chat-hf_zeroshot_temperature_0.0.pkl')
df2 = pd.read_pickle('/home/amey/depository/cs-eval/predictions_rebuttal/meta-llama-Llama-3-8b-chat-hf_zeroshot_temperature_0.5.pkl')
df3 = pd.read_pickle('/home/amey/depository/cs-eval/predictions_rebuttal/meta-llama-Llama-3-8b-chat-hf_zeroshot_temperature_1.pkl')

prediction_cols = [col for col in df1.columns if 'prediction_' in col]

for col in prediction_cols:
    df1 = df1[df1[col].notna()]
    df1[col] = df1[col].apply(lambda x: extract_content_from_response(x))
    df1[col] = df1[col].apply(lambda x: extract_score(x))
    df1 = df1.rename(columns={col: col.replace('meta-llama/Llama-3-8b-chat-hf','')})

    df2 = df2[df2[col].notna()]
    df2[col] = df2[col].apply(lambda x: extract_content_from_response(x))
    df2[col] = df2[col].apply(lambda x: extract_score(x))
    df2 = df2.rename(columns={col: col.replace('meta-llama/Llama-3-8b-chat-hf','')})
    
    df3 = df3[df3[col].notna()]
    df3[col] = df3[col].apply(lambda x: extract_content_from_response(x))
    df3[col] = df3[col].apply(lambda x: extract_score(x))
    df3 = df3.rename(columns={col: col.replace('meta-llama/Llama-3-8b-chat-hf','')})

prediction_cols = [col for col in df1.columns if 'prediction_' in col]
# df_analysis = pd.concat([df1[prediction_cols], df2[prediction_cols], df3[prediction_cols]], axis=1)
df_analysis = pd.concat([df1[prediction_cols], df2[prediction_cols], df3[prediction_cols]], axis=1)
df_analysis = df_analysis[:1300]
df_analysis = df_analysis.dropna()
print(df_analysis.shape)
df_analysis.head()

(1300, 12)


,prediction__relevance_score,prediction__coherence_score,prediction__suitableness_score,prediction__aggressiveness_score,prediction__relevance_score,prediction__coherence_score,prediction__suitableness_score,prediction__aggressiveness_score,prediction__relevance_score,prediction__coherence_score,prediction__suitableness_score,prediction__aggressiveness_score
0,5.0,4.0,2.0,1.0,5.0,4.0,2.0,1.0,5.0,4.0,2.0,1.0
1,5.0,4.0,2.0,2.0,5.0,4.0,2.0,1.0,4.0,4.0,2.0,1.0
2,5.0,4.0,2.0,2.0,5.0,4.0,2.0,2.0,4.0,4.0,2.0,1.0
3,1.0,1.0,1.0,4.0,1.0,1.0,2.0,4.0,2.0,1.0,2.0,4.0
4,2.0,2.0,1.0,2.0,1.0,2.0,1.0,2.0,3.0,2.0,2.0,2.0


In [103]:
for col in prediction_cols:
    print(col)
    print(f"Standard deviation: {df_analysis[col].std(axis=1).mean()}")

prediction__relevance_score
Standard deviation: 0.22998096797735215
prediction__coherence_score
Standard deviation: 0.170563729426318
prediction__suitableness_score
Standard deviation: 0.2749732399823688
prediction__aggressiveness_score
Standard deviation: 0.2793979324154946


In [10]:
import glob
from utils import postprocess_llm_response, extract_content_from_response

df1 = pd.read_pickle('/home/amey/depository/cs-eval/predictions_rebuttal/mistralai-Mistral-7B-Instruct-v0.3_zeroshot_temperature_0.0.pkl')
df2 = pd.read_pickle('/home/amey/depository/cs-eval/predictions_rebuttal/mistralai-Mistral-7B-Instruct-v0.3_zeroshot_temperature_0.5.pkl')
df3 = pd.read_pickle('/home/amey/depository/cs-eval/predictions_rebuttal/mistralai-Mistral-7B-Instruct-v0.3_zeroshot_temperature_1.pkl')

prediction_cols = [col for col in df1.columns if 'prediction_' in col]

for col in prediction_cols:
    df1 = df1[df1[col].notna()]
    # print(df1.shape)
    df1[col] = df1[col].apply(lambda x: extract_content_from_response(x))
    df1[col] = df1[col].apply(lambda x: extract_score(x))
    df1 = df1.rename(columns={col: col.replace('mistralai/Mistral-7B-Instruct-v0.3_','')})

    df2 = df2[df2[col].notna()]
    # print(df2.shape)
    df2[col] = df2[col].apply(lambda x: extract_content_from_response(x))
    df2[col] = df2[col].apply(lambda x: extract_score(x))
    df2 = df2.rename(columns={col: col.replace('mistralai/Mistral-7B-Instruct-v0.3_','')})
    
    df3 = df3[df3[col].notna()]
    # print(df3.shape)
    df3[col] = df3[col].apply(lambda x: extract_content_from_response(x))
    df3[col] = df3[col].apply(lambda x: extract_score(x))
    df3 = df3.rename(columns={col: col.replace('mistralai/Mistral-7B-Instruct-v0.3_','')})

prediction_cols = [col for col in df1.columns if 'prediction_' in col]
# df_analysis = pd.concat([df1[prediction_cols], df2[prediction_cols], df3[prediction_cols]], axis=1)
df_analysis = pd.concat([df1[prediction_cols], df2[prediction_cols], df3[prediction_cols]], axis=1)
df_analysis = df_analysis[:1300]
df_analysis = df_analysis.dropna()
print(df_analysis.shape)
df_analysis.head()

(1300, 12)


,prediction_relevance_score,prediction_coherence_score,prediction_suitableness_score,prediction_aggressiveness_score,prediction_relevance_score,prediction_coherence_score,prediction_suitableness_score,prediction_aggressiveness_score,prediction_relevance_score,prediction_coherence_score,prediction_suitableness_score,prediction_aggressiveness_score
0,5.0,4.0,2.0,2.0,5.0,4.0,3.0,1.0,5.0,3.0,1.0,1.0
1,5.0,4.0,3.0,2.0,5.0,4.0,3.0,1.0,5.0,4.0,3.0,1.0
2,5.0,5.0,3.0,2.0,5.0,5.0,3.0,2.0,5.0,4.0,3.0,2.0
3,1.0,1.0,1.0,4.0,1.0,1.0,1.0,4.0,2.0,1.0,1.0,4.0
4,5.0,2.0,1.0,1.0,5.0,2.0,1.0,1.0,5.0,2.0,1.0,1.0


In [5]:
for col in prediction_cols:
    print(col)
    print(f"Standard deviation: {df_analysis[col].std(axis=1).mean()}")

prediction_relevance_score
Standard deviation: 0.09370838984539312
prediction_coherence_score
Standard deviation: 0.14022152655810963
prediction_suitableness_score
Standard deviation: 0.1043905062653532
prediction_aggressiveness_score
Standard deviation: 0.14999206957516478


In [20]:
for col in prediction_cols:
    print(col)
    values = df_analysis[col].values.tolist()
    values1 = [x[0] for x in values]
    values2 = [x[1] for x in values]
    values3 = [x[2] for x in values]

    h_statistic, p_value = stats.kruskal(values1, values2, values3)
    print(f"Kruskal-Wallis H-statistic: {h_statistic}, p-value: {p_value}")

prediction_relevance_score
Kruskal-Wallis H-statistic: 2.8135023553270098, p-value: 0.24493775114894545
prediction_coherence_score
Kruskal-Wallis H-statistic: 0.8002685402755404, p-value: 0.6702300481128598
prediction_suitableness_score
Kruskal-Wallis H-statistic: 1.8396297907360921, p-value: 0.39859281563273546
prediction_aggressiveness_score
Kruskal-Wallis H-statistic: 0.38873138618738207, p-value: 0.8233567533291871


In [15]:
df_analysis['prediction_coherence_score'].values.tolist()

[[4.0, 4.0, 3.0],
 [4.0, 4.0, 4.0],
 [5.0, 5.0, 4.0],
 [1.0, 1.0, 1.0],
 [2.0, 2.0, 2.0],
 [4.0, 4.0, 4.0],
 [3.0, 4.0, 3.0],
 [5.0, 5.0, 5.0],
 [4.0, 4.0, 4.0],
 [3.0, 4.0, 4.0],
 [4.0, 4.0, 4.0],
 [4.0, 4.0, 4.0],
 [4.0, 4.0, 5.0],
 [4.0, 4.0, 4.0],
 [4.0, 4.0, 4.0],
 [4.0, 4.0, 4.0],
 [4.0, 4.0, 4.0],
 [5.0, 5.0, 5.0],
 [4.0, 4.0, 4.0],
 [4.0, 4.0, 4.0],
 [2.0, 3.0, 2.0],
 [4.0, 4.0, 4.0],
 [3.0, 3.0, 3.0],
 [4.0, 4.0, 4.0],
 [5.0, 5.0, 5.0],
 [4.0, 4.0, 4.0],
 [4.0, 4.0, 4.0],
 [4.0, 4.0, 4.0],
 [2.0, 2.0, 2.0],
 [4.0, 4.0, 4.0],
 [4.0, 4.0, 4.0],
 [4.0, 4.0, 4.0],
 [5.0, 5.0, 5.0],
 [3.0, 3.0, 3.0],
 [4.0, 4.0, 4.0],
 [4.0, 4.0, 4.0],
 [3.0, 3.0, 3.0],
 [4.0, 4.0, 4.0],
 [5.0, 5.0, 5.0],
 [5.0, 5.0, 5.0],
 [2.0, 2.0, 2.0],
 [4.0, 4.0, 4.0],
 [4.0, 4.0, 4.0],
 [3.0, 3.0, 3.0],
 [3.0, 3.0, 3.0],
 [4.0, 4.0, 4.0],
 [5.0, 5.0, 5.0],
 [5.0, 5.0, 5.0],
 [5.0, 5.0, 4.0],
 [5.0, 5.0, 5.0],
 [4.0, 4.0, 4.0],
 [3.0, 3.0, 3.0],
 [5.0, 5.0, 5.0],
 [3.0, 4.0, 3.0],
 [4.0, 4.0, 5.0],
 [4.0, 4.0

In [19]:
values = df_analysis['prediction_coherence_score'].values.tolist()
values1 = [x[0] for x in values]
values2 = [x[1] for x in values]
values3 = [x[2] for x in values]

h_statistic, p_value = stats.kruskal(values1, values2, values3)
print(f"Kruskal-Wallis H-statistic: {h_statistic}, p-value: {p_value}")

Kruskal-Wallis H-statistic: 0.8002685402755404, p-value: 0.6702300481128598


### New paired t-tests

In [1]:
import numpy as np
from scipy.stats import ttest_rel

def paired_t_test(ratings_A, ratings_B):
    # Perform the paired t-test
    t_stat, p_value = ttest_rel(ratings_A, ratings_B)
    
    return t_stat, p_value

# Example ratings for models A and B
ratings_A = [4, 3, 5, 2, 4, 3, 4, 2, 5, 3]
ratings_B = [3, 2, 4, 1, 3, 2, 3, 1, 4, 2]

# Perform the paired t-test
t_stat, p_value = paired_t_test(ratings_A, ratings_B)

# Print the results
print(f"t-statistic: {t_stat:.4f}")
print(f"P-value: {p_value:.4f}")

# Decision based on p-value
alpha = 0.05
if p_value < alpha:
    print("Reject the null hypothesis: The difference is statistically significant.")
else:
    print("Fail to reject the null hypothesis: The difference is not statistically significant.")

t-statistic: inf
P-value: 0.0000
Reject the null hypothesis: The difference is statistically significant.


/home/amey/miniconda3/envs/socionet/lib/python3.9/site-packages/scipy/stats/_axis_nan_policy.py:531: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)


In [4]:
df_results = pd.read_csv('/home/amey/depository/cs-eval/data/annotations/dataset_metrics_calculated_renamecols.csv')
df_results.head(1)

,level_0,index,hatespeech,counterspeech,predicted_counterspeech,csType,suitableness_score,relevance_score,coherence_score,aggressiveness_score,...,zs_Llama-3-8b-chat-hf_suitableness_score,zs_Llama-3-8b-chat-hf_aggressiveness_score,Mistral-7B-Instruct-v03_relevance_score,Mistral-7B-Instruct-v03_coherence_score,Mistral-7B-Instruct-v03_suitableness_score,Mistral-7B-Instruct-v03_aggressiveness_score,zs_Mistral-7B-Instruct-v03_relevance_score,zs_Mistral-7B-Instruct-v03_coherence_score,zs_Mistral-7B-Instruct-v03_suitableness_score,zs_Mistral-7B-Instruct-v03_aggressiveness_score
0,0,0.0,Jews have no place here. They are not a part o...,"Jews are part of most societies, why should we...",Why do you believe that jews are not part of t...,Questioning,1.666667,3.333333,2.333333,1.666667,...,3.0,1.0,-1.0,-1.0,-1.0,-1.0,5.0,3.0,3.0,2.0


In [6]:
for col in df_results.columns:
    print(col)

level_0
index
hatespeech
counterspeech
predicted_counterspeech
csType
suitableness_score
relevance_score
coherence_score
aggressiveness_score
annotator
source
prediction_(prompt_aggressiveness_score)_(gpt-4)
prediction_(prompt_relevance_score)_(gpt-4)
uuid
prediction_(prompt_aggressiveness_score)_(gpt3.5-turbo)
prediction_(prompt_coherence_score)_(gpt-3.5-turbo)
prediction_(prompt_relevance_score)_(gpt-3.5-turbo)
prediction_(prompt_suitableness_score)_(gpt-3.5-turbo)
bert_score_(pred_cs, cs)
bm25_score_(pred_cs, cs)
obscenity_(pred_cs)
identity_attack_(pred_cs)
insult_(pred_cs)
bleu_1_(pred_cs, cs)
bleu_2_(pred_cs, cs)
rouge_l_(pred_cs, cs)
rouge_1_(pred_cs, cs)
rouge_2_(pred_cs, cs)
meteor_score_(pred_cs, cs)
bert_score_(hs, pred_cs)
toxicity_(pred_cs)
pc_score_(hs, pred_cs)
cd_score_(hs, pred_cs)
aq_score_(pred_cs)
bm25_score_(hs, pred_cs)
mauve_score_(hs, pred_cs)
mauve_score_(pred_cs, cs)
coherence_UniEval
relevance_UniEval
aggressiveness_UniEval
suitableness_UniEval
prompt_relevan

In [12]:
aspect_name = 'gpt-4_relevance_score'

ratings_A = df_results['gpt-4_relevance_score'].values.tolist()
ratings_B = df_results['GEVAL-4_relevance_score'].values.tolist()


ratings_A = [x for x in ratings_A if x > -1]
ratings_B = [x for x in ratings_A if x > -1]

# Perform the paired t-test
t_stat, p_value = paired_t_test(ratings_A, ratings_B)

# Print the results
print(f"t-statistic: {t_stat:.4f}")
print(f"P-value: {p_value:.4f}")

# Decision based on p-value
alpha = 0.05
if p_value < alpha:
    print("Reject the null hypothesis: The difference is statistically significant.")
else:
    print("Fail to reject the null hypothesis: The difference is not statistically significant.")

t-statistic: nan
P-value: nan
Fail to reject the null hypothesis: The difference is not statistically significant.
